## Import Modules and Setup Environment

In [1]:
import polars as pl
from dotenv import load_dotenv
from IPython.display import display_markdown

load_dotenv()

from argmap.dataModel import DataModel, Comments, Topics, HierarchicalTopics, Arguments, ArgumentCommentMap, Summary

## Load Dataset

In [2]:
DATASET = "american-assembly.bowling-green"

summary = Summary(DATASET)
summary.description = summary.get('conversation-description')

comments = Comments(DATASET).load_from_parquet()
topics = Topics(DATASET).load_from_parquet()
hierarchicalTopics = HierarchicalTopics(DATASET).load_from_parquet()
arguments = Arguments(DATASET).load_from_parquet()
argumentCommentMap = ArgumentCommentMap(DATASET, table='argumentCommentMap').load_from_parquet()

argumentTopicSupport = DataModel(DATASET, table='argumentTopicSupport').load_from_parquet()

argumentTopicSupport.glimpse()

Rows: 102
Columns: 11
$ topicId               <i16> 10, 10, 10, 10, 10, 10, 1, 1, 6, 1
$ argumentId            <u16> 3, 4, 0, 1, 2, 5, 5, 4, 4, 0
$ support               <i32> 1065, 1056, 1058, 856, 908, 891, 1091, 1109, 765, 1115
$ agrees                <u32> 1129, 1120, 1123, 912, 971, 953, 1182, 1204, 836, 1220
$ disagrees             <u32> 64, 64, 65, 56, 63, 62, 91, 95, 71, 105
$ agreeability          <f64> 0.9463537300922046, 0.9459459459459459, 0.9452861952861953, 0.9421487603305785, 0.9390715667311412, 0.9389162561576355, 0.9285153181461115, 0.9268668206312548, 0.9217199558985667, 0.9207547169811321
$ argumentTitle         <str> 'Driving Competition and Improving Services', 'Improving Internet Speeds and Competition', 'Driving Competition and Improving Internet Access', 'Ensuring Fair Cable TV Services', 'Bridging the Digital Divide', 'Bridging the Digital Divide through Free WiFi', 'Improved Traffic Flow', 'Improved Access and Service Roads', 'Comprehensive Sex Education', 'Im

## Argdown Template

```argdown-map
===
sourceHighlighter:
    removeFrontMatter: true
webComponent:
    withoutMaximize: true
model:
    removeTagsFromText: true
selection:
    excludeDisconnected: false
color:
    tagColors:
        Human: "#99cc99"
        AI: "#9999cc"
===

[statement]: I am a statement. #Human
 - <argument 1>
 + <argument 2>
    <- <argument 3>

 # I am a group

<argument 1>: I am an attacking argument. #AI

<argument 2>: I am a supporting argument. #AI

## I am a subgroup

<argument 3>: I am an attacked argument. #AI
```

## Argdown Grammar Specification

In the following block, we define the grammar for the Argdown syntax. This is a simplified version of the original grammar from the [Argdown documentation](https://argdown.org/syntax/).

In [3]:
def argdown_heading(label, content=None):
    if content is None:
        return f"\n# {label}\n"
    else:
        return f"\n# {label}\n\n{content}\n"


def argdown_topic(topicName, topicContent, supports=None, tag="AI"):
    emoji = "🤖" if tag == "AI" else "🙂"
    output = f"\n[{topicName}]: {emoji} {topicContent} #{tag}\n"
    if supports is not None:
        return output + argdown_supports(supports)
    return output


def argdown_argument(label, content, supports=None, supportsArgument=None, tag="AI"):
    emoji = "🤖" if tag == "AI" else "🙂"
    output = f"\n<{label}>: {emoji} {content} #{tag}\n"
    if supports is not None:
        return output + argdown_supports(supports)
    if supportsArgument is not None:
        return output + argdown_supports(supportsArgument, argument=True)
    return output


def argdown_comment(label, comment, supports=None, supportsArgument=None, tag="Human"):
    emoji = "🤖" if tag == "AI" else "🙂"
    output = f"\n[{label}]: {emoji} {comment} #{tag}\n"
    if supports is not None:
        return output + argdown_supports(supports)
    if supportsArgument is not None:
        return output + argdown_supports(supportsArgument, argument=True)
    return output


def argdown_supported_by(label):
    return f"  + [{label}]\n"


def argdown_refuted_by(label):
    return f"  - [{label}]\n"


def argdown_supports(label, argument=False):
    if argument:
        return f"  +> <{label}>\n"
    return f"  +> [{label}]\n"


def argdown_refutes(label):
    return f"  -> [{label}]\n"


def argdown_markdown_template(output):
    return f"""\
```argdown-map
===
sourceHighlighter:
    removeFrontMatter: true
webComponent:
    withoutMaximize: true
    height: 640px
model:
    removeTagsFromText: true
# map:
#     statementLabelMode: text
#     argumentLabelMode: text
selection:
    excludeDisconnected: false
color:
    tagColors:
        Human: "#99cc99"
        AI: "#9999cc"
===

{output}
```
"""

## Argument Map for Top Insights and Comments

For each topic, we display the top 3 arguments by the agreeability ratio. Additionally, we show the three most agreed-upon comments for each of those arguments based on raw `agree` votes.

In [4]:
TOP_N_ARGUMENTS = 3
TOP_N_COMMENTS = 3

# output discussion title and question
output = argdown_heading(summary.topic, summary.description)
# output += argdown_comment(summary.topic, summary.description)

# argumentTopicSupport is a view that joins arguments and topics, and adds support and agreeability scores

# filter to only the top argument for each topic
topArguments = (
    argumentTopicSupport.df.lazy()
    .sort('agreeability', descending=True)
    .group_by('topicId', maintain_order=True).head(TOP_N_ARGUMENTS)
).collect()


# output each topic once
# get unique topicId from argumentTopicSupport

topicIds = argumentTopicSupport.df.get_column(
    'topicId').unique().sort().to_list()

for topicId in topicIds:
    topic = topics.get(topicId)
    heading = topic['Heading']
    title = topic['Title']
    output += argdown_topic(heading, title)

    # output += argdown_supports()

# link topic to discussion title?

# output top-n arguments and link to topic

for topicId, topicHeading, argumentId, argumentTitle, argumentContent, agreeability, agrees, disagrees in (
    topArguments
    .select('topicId', 'topicHeading', 'argumentId', 'argumentTitle', 'argumentContent', 'agreeability', 'agrees', 'disagrees')
        .iter_rows()):

    body = f"{argumentContent} _({agrees + disagrees} voters approximated, {agreeability*100:.1f}% tend to agree_)"
    output += argdown_argument(argumentTitle, body, topicHeading)

# find the top `n` comments for each argument

argumentCommentMapSupport = argumentCommentMap.df.filter(
    relationship='SUPPORT')
topArgumentComments = (
    topArguments
    .select('topicId', 'argumentId', 'argumentTitle', 'thoughts')
    .join(argumentCommentMapSupport, on=['argumentId', 'topicId'], how='left')
    .join(comments.df.select('commentId', 'commentText', 'agrees', 'disagrees', agreeability=(pl.col('agrees')/(pl.col('agrees')+pl.col('disagrees')))), on='commentId', how='left')
    .sort('agrees', descending=True)
    .group_by('topicId', 'argumentId', maintain_order=True)
    .head(TOP_N_COMMENTS)
)

for topicId, argumentId, argumentTitle, thoughts, commentId, commentText, agrees, disagrees, agreeability in (
    topArgumentComments
    .select('topicId', 'argumentId', 'argumentTitle', 'thoughts', 'commentId', 'commentText', 'agrees', 'disagrees', 'agreeability')
        .iter_rows()):

    body = f"{commentText} _({agrees + disagrees} votes, {agreeability*100:.1f}% agree_)"
    output += argdown_comment(f'Comment {commentId}', body, supportsArgument=argumentTitle)

display_markdown(argdown_markdown_template(output), raw=True)

```argdown-map
===
sourceHighlighter:
    removeFrontMatter: true
webComponent:
    withoutMaximize: true
    height: 640px
model:
    removeTagsFromText: true
# map:
#     statementLabelMode: text
#     argumentLabelMode: text
selection:
    excludeDisconnected: false
color:
    tagColors:
        Human: "#99cc99"
        AI: "#9999cc"
===


# Improving Bowling Green / Warren County

What do you believe should change in Bowling Green/Warren County in order to make it a better place to live, work and spend time?

[Addressing Homelessness with Occupational Taxes]: 🤖 Implementing Occupational Taxes to Address Homelessness #AI

[Improving Traffic Flow]: 🤖 Implementation of Traffic Control Measures to Improve Road Flow #AI

[No to Fairness Ordinance]: 🤖 Opposition to the Proposed Fairness Ordinance in Bowling Green #AI

[College Campus Improvements]: 🤖 Implementing a Comprehensive Parking and Food Assistance Program for College Students on Campus #AI

[Improving City Planning and Zoning]: 🤖 Implementing a Comprehensive City Planning and Zoning Strategy for Better Neighborhoods and Community Activities #AI

[Opioid Epidemic and Healthcare]: 🤖 The Role of Doctors and Teenagers in the Opioid Crisis #AI

[Public School Funding and Accountability]: 🤖 The Importance of Adequate Funding and Accountability in Public School Education #AI

[Community Enrichment Hub]: 🤖 Bowling Green Community Enrichment Center: A Multi-Purpose Hub for Sports, Arts, and Learning #AI

[Pedestrian-Friendly Infrastructure]: 🤖 Implementation of Pedestrian-Friendly Infrastructure in Key Areas #AI

[Enhancing Community Safety and Tourism]: 🤖 Implementing Water and Fire Protection Measures in Warren County's Tuckertown Area #AI

[Internet Choices and Competition]: 🤖 Expansion of Residential Internet Services in Bowling Green #AI

[Marijuana Legalization]: 🤖 Legalized Marijuana: A Revenue-Generating Solution for Bowling Green #AI

[Enhancing Recycling and Waste Management]: 🤖 Implementing Improved Recycling and Waste Management Services in Bowling Green #AI

[Improved Transit Services]: 🤖 Enhanced Public Transportation System for Accessible Daily Commute #AI

<Driving Competition and Improving Services>: 🤖 Increasing competition for cable and internet companies will drive down rates, improve services, and provide more fiber options for Bowling Green residents. _(1193 voters approximated, 94.6% tend to agree_) #AI
  +> [Internet Choices and Competition]

<Improving Internet Speeds and Competition>: 🤖 Making fiber internet options available will improve internet speeds, drive down rates, and promote competition for cable and internet companies in Bowling Green. _(1184 voters approximated, 94.6% tend to agree_) #AI
  +> [Internet Choices and Competition]

<Driving Competition and Improving Internet Access>: 🤖 The expansion of residential internet services, including the provision of fiber options, will promote competition, drive down rates, and improve internet speeds and accessibility for Bowling Green residents. _(1188 voters approximated, 94.5% tend to agree_) #AI
  +> [Internet Choices and Competition]

<Improved Traffic Flow>: 🤖 Reducing the number of traffic lights and implementing roundabouts can significantly improve traffic flow, ensuring a smoother driving experience for drivers. _(1273 voters approximated, 92.9% tend to agree_) #AI
  +> [Improving Traffic Flow]

<Improved Access and Service Roads>: 🤖 Proper planning and improvement of service roads and access roads can significantly improve safety and convenience for drivers, reducing traffic jams and ensuring a smoother driving experience. _(1299 voters approximated, 92.7% tend to agree_) #AI
  +> [Improving Traffic Flow]

<Improved Traffic Control Measures>: 🤖 Implementing traffic control measures such as roundabouts, traffic lights, and turn lanes can significantly improve road flow and safety, reducing traffic congestion and ensuring a smoother driving experience. _(1325 voters approximated, 92.1% tend to agree_) #AI
  +> [Improving Traffic Flow]

<Comprehensive Sex Education>: 🤖 Comprehensive sexual education is necessary to lower STD and teen pregnancy rates. _(907 voters approximated, 92.2% tend to agree_) #AI
  +> [Public School Funding and Accountability]

<Physical Activity for Health>: 🤖 Daily physical activity in schools is vital to combating childhood obesity and promoting healthy lifestyles. _(529 voters approximated, 90.0% tend to agree_) #AI
  +> [Public School Funding and Accountability]

<Essential Education Funding>: 🤖 Increased funding for public schools, especially art education programs, is essential to fostering a well-rounded education for students. _(1217 voters approximated, 89.2% tend to agree_) #AI
  +> [Public School Funding and Accountability]

<Deter Littering>: 🤖 Implementing fines can deter littering and illegal dumping, promoting a cleaner and safer environment. _(690 voters approximated, 91.2% tend to agree_) #AI
  +> [Enhancing Recycling and Waste Management]

<Clean Yards>: 🤖 Enforcing cleanliness and reducing junk cars can significantly improve the appearance of Bowling Green. _(690 voters approximated, 91.2% tend to agree_) #AI
  +> [Enhancing Recycling and Waste Management]

<Responsible Businesses>: 🤖 Requiring businesses to take responsibility for litter can significantly reduce waste and promote a cleaner environment. _(796 voters approximated, 90.6% tend to agree_) #AI
  +> [Enhancing Recycling and Waste Management]

<Term Limits for Fair Representation>: 🤖 Implement term limits for local elected officials to promote fair representation, prevent misuse of power, and encourage fresh perspectives. _(951 voters approximated, 90.5% tend to agree_) #AI
  +> [Addressing Homelessness with Occupational Taxes]

<Diversity in City/County Government>: 🤖 Promote racial and ethnic diversity in city/county government to enhance representation, recruitment, and growth. _(1329 voters approximated, 84.8% tend to agree_) #AI
  +> [Addressing Homelessness with Occupational Taxes]

<Police Presence in County Areas>: 🤖 Increase police presence and resources in county areas to ensure public safety and address domestic violence disputes effectively. _(731 voters approximated, 84.1% tend to agree_) #AI
  +> [Addressing Homelessness with Occupational Taxes]

<Modernizing Emergency Services>: 🤖 Upgrading volunteer fire departments will better equip them to serve the growing population and ensure the safety of Warren County residents. _(694 voters approximated, 90.2% tend to agree_) #AI
  +> [Enhancing Community Safety and Tourism]

<Enhancing Safety and Quality of Life>: 🤖 Extending water lines and implementing fire protection in the Tuckertown area will significantly improve safety and quality of life for residents, while reducing insurance costs. _(808 voters approximated, 89.2% tend to agree_) #AI
  +> [Enhancing Community Safety and Tourism]

<Boosting Tourism and Economy>: 🤖 Implementing a marketing campaign and promoting diverse tourism activities will attract more visitors, boosting the local economy and enhancing Warren County's appeal. _(817 voters approximated, 87.9% tend to agree_) #AI
  +> [Enhancing Community Safety and Tourism]

<Historic Preservation and Unique Architectural Heritage>: 🤖 Preserving historic homes and buildings, and maintaining unique architectural heritage are essential for fostering a strong sense of community, celebrating Bowling Green's rich history, and enhancing the city's overall appeal. _(1209 voters approximated, 89.4% tend to agree_) #AI
  +> [Improving City Planning and Zoning]

<Equitable Resource Distribution>: 🤖 Ensuring equitable distribution of resources across all areas of the city is vital for fostering a more inclusive and thriving community, where every resident has access to essential amenities and opportunities. _(1368 voters approximated, 88.9% tend to agree_) #AI
  +> [Improving City Planning and Zoning]

<Local Business Development and Support>: 🤖 Encouraging local business development and supporting small businesses is essential for Bowling Green's unique character, as it fosters a stronger sense of community and boosts the local economy. _(1335 voters approximated, 88.7% tend to agree_) #AI
  +> [Improving City Planning and Zoning]

<Enhancing Aesthetic Appeal>: 🤖 Enforcing zoning mandates will improve the aesthetic appeal of Bowling Green and maintain property values. _(936 voters approximated, 88.6% tend to agree_) #AI
  +> [No to Fairness Ordinance]

<Enforcing Noise Ordinances>: 🤖 Enforcing noise ordinances and codes will maintain peace and order in residential areas, ensuring a more enjoyable living experience for all Bowling Green residents. _(802 voters approximated, 82.8% tend to agree_) #AI
  +> [No to Fairness Ordinance]

<Empowering Blue Collar Workers>: 🤖 Investing in the North-Northeast ends of town will empower blue collar workers and contribute to a more equitable Bowling Green community. _(995 voters approximated, 82.2% tend to agree_) #AI
  +> [No to Fairness Ordinance]

<Improving Downtown Pedestrian Experience>: 🤖 Making downtown a pedestrian-friendly mall can significantly enhance the overall pedestrian experience and promote local businesses. _(1204 voters approximated, 86.5% tend to agree_) #AI
  +> [Pedestrian-Friendly Infrastructure]

<Improving Pedestrian Safety>: 🤖 Installing sidewalks in areas with heavy pedestrian traffic can significantly improve pedestrian safety and accessibility. _(1213 voters approximated, 86.2% tend to agree_) #AI
  +> [Pedestrian-Friendly Infrastructure]

<Enhancing Greenway Experience>: 🤖 A continuous and connected Greenway trail system can significantly enhance the overall pedestrian and recreational experience. _(1261 voters approximated, 85.9% tend to agree_) #AI
  +> [Pedestrian-Friendly Infrastructure]

<Parking Regulation Enforcement>: 🤖 Monitoring and enforcing parking regulations and handicap spaces will ensure fair access to parking and promote safety within the community. _(718 voters approximated, 85.9% tend to agree_) #AI
  +> [College Campus Improvements]

<Downtown Parking Improvement>: 🤖 Improving downtown parking and accessibility by increasing parking spaces, making reserved spaces available during evenings, and enforcing parking regulations will benefit existing businesses and their customers. _(762 voters approximated, 85.4% tend to agree_) #AI
  +> [College Campus Improvements]

<Parking Program Improvement>: 🤖 Implementing a comprehensive parking program with garages, green spaces, and connected lots near WKU campus will significantly improve parking accessibility and enforcement for college students. _(852 voters approximated, 83.7% tend to agree_) #AI
  +> [College Campus Improvements]

<Improved daily commuting>: 🤖 Expanding the public transportation system's service area, increasing accessibility, and adding more stops in residential areas will greatly improve daily commuting for those without cars. _(1064 voters approximated, 85.6% tend to agree_) #AI
  +> [Improved Transit Services]

<Improved Nashville Airport transport>: 🤖 Providing regular and affordable daily transport to Nashville Airport will greatly benefit both residents and visitors. _(1064 voters approximated, 85.6% tend to agree_) #AI
  +> [Improved Transit Services]

<Real bus system>: 🤖 Implementing a real bus system will significantly improve public transportation in the area. _(1037 voters approximated, 85.3% tend to agree_) #AI
  +> [Improved Transit Services]

<Educated and Cultured Community>: 🤖 Increasing the number of cultural and educational establishments would promote learning, personal growth, and community development. _(1285 voters approximated, 85.5% tend to agree_) #AI
  +> [Community Enrichment Hub]

<Engaged University-Community Partnership>: 🤖 Fostering cooperation between WKU and Bowling Green would enhance learning experiences, promote community engagement, and prepare students for their future careers. _(1141 voters approximated, 84.4% tend to agree_) #AI
  +> [Community Enrichment Hub]

<Culturally Enriched City>: 🤖 Engaging art installations would enrich Bowling Green's cultural landscape, fostering creativity and community pride. _(1156 voters approximated, 84.3% tend to agree_) #AI
  +> [Community Enrichment Hub]

<Improve medical care access>: 🤖 Ensuring timely access to medical care, including appointments and healthcare facilities, is crucial to alleviate suffering and prevent health conditions from worsening. _(937 voters approximated, 83.4% tend to agree_) #AI
  +> [Opioid Epidemic and Healthcare]

<Implement stricter opioid regulations>: 🤖 Enacting stricter regulations for opioid drugs and expanding mental health treatment access will help mitigate the opioid crisis and its devastating consequences. _(1053 voters approximated, 81.2% tend to agree_) #AI
  +> [Opioid Epidemic and Healthcare]

<Educate about drug risks>: 🤖 Educating students about drug risks and side effects through drug information classes and school curricula will empower them to make informed decisions and reduce drug use. _(846 voters approximated, 79.7% tend to agree_) #AI
  +> [Opioid Epidemic and Healthcare]

<Healthier Environment>: 🤖 Implement policies to prohibit smoking in public parks and within 25 feet of businesses to address the concerns of 70% and 68% of voters, respectively, promoting a healthier environment. _(327 voters approximated, 71.3% tend to agree_) #AI
  +> [Marijuana Legalization]

<Voter Support>: 🤖 Pass a resolution in support of medical cannabis to address the needs of 60% of voters and enhance the quality of life in Bowling Green. _(1000 voters approximated, 65.8% tend to agree_) #AI
  +> [Marijuana Legalization]

<Economic Growth>: 🤖 Re-legalize industrial hemp in Kentucky to address the needs of 79% of voters, promoting economic growth and job opportunities. _(909 voters approximated, 63.4% tend to agree_) #AI
  +> [Marijuana Legalization]

[Comment 21]: 🙂 The arts are an important component of K-12 education. _(787 votes, 90.0% agree_) #Human
  +> <Essential Education Funding>

[Comment 39]: 🙂 All students in Warren county should have the best available education, not the Kentucky minimum, at all times. _(599 votes, 94.2% agree_) #Human
  +> <Essential Education Funding>

[Comment 38]: 🙂 All schools in the county and city should work together to provide the same minimum level of education to students. _(487 votes, 88.3% agree_) #Human
  +> <Essential Education Funding>

[Comment 10]: 🙂 In order to better combat the opioid epidemic, it is time to view drug addiction as a health problem rather than a criminal justice problem. _(839 votes, 79.5% agree_) #Human
  +> <Improve medical care access>

[Comment 336]: 🙂 With drug use being so prevalent in Warren county, better treatment options need to be offered. _(358 votes, 86.9% agree_) #Human
  +> <Improve medical care access>

[Comment 476]: 🙂 We have an estimated 125,000 people but we still don't have a hospital that is trauma center and have to ship everyone to TN or Louisville. _(154 votes, 87.0% agree_) #Human
  +> <Improve medical care access>

[Comment 10]: 🙂 In order to better combat the opioid epidemic, it is time to view drug addiction as a health problem rather than a criminal justice problem. _(839 votes, 79.5% agree_) #Human
  +> <Implement stricter opioid regulations>

[Comment 11]: 🙂 Law enforcement should send drug possession arrestees for rehabilitation services rather than imprisonment. _(719 votes, 71.8% agree_) #Human
  +> <Implement stricter opioid regulations>

[Comment 336]: 🙂 With drug use being so prevalent in Warren county, better treatment options need to be offered. _(358 votes, 86.9% agree_) #Human
  +> <Implement stricter opioid regulations>

[Comment 10]: 🙂 In order to better combat the opioid epidemic, it is time to view drug addiction as a health problem rather than a criminal justice problem. _(839 votes, 79.5% agree_) #Human
  +> <Educate about drug risks>

[Comment 742]: 🙂 There should be more effort put into teaching students about any potential harmful side effects of drugs, illegal or otherwise. _(87 votes, 82.8% agree_) #Human
  +> <Educate about drug risks>

[Comment 709]: 🙂 Drug information classes should be given to incoming freshmen of WKU to educate them about the risks and health hazards. _(97 votes, 62.9% agree_) #Human
  +> <Educate about drug risks>

[Comment 47]: 🙂 Continue downtown development to the river.  Tourists always seek out parks & restaurants near water. _(747 votes, 89.2% agree_) #Human
  +> <Improving Downtown Pedestrian Experience>

[Comment 214]: 🙂 More bike trails and walking trails would be a great addition. _(613 votes, 85.8% agree_) #Human
  +> <Improving Downtown Pedestrian Experience>

[Comment 146]: 🙂 Our sidewalks need more connectivity. It's too piecemealed and makes some areas unusable. _(510 votes, 87.6% agree_) #Human
  +> <Improving Downtown Pedestrian Experience>

[Comment 47]: 🙂 Continue downtown development to the river.  Tourists always seek out parks & restaurants near water. _(747 votes, 89.2% agree_) #Human
  +> <Improving Pedestrian Safety>

[Comment 214]: 🙂 More bike trails and walking trails would be a great addition. _(613 votes, 85.8% agree_) #Human
  +> <Improving Pedestrian Safety>

[Comment 146]: 🙂 Our sidewalks need more connectivity. It's too piecemealed and makes some areas unusable. _(510 votes, 87.6% agree_) #Human
  +> <Improving Pedestrian Safety>

[Comment 47]: 🙂 Continue downtown development to the river.  Tourists always seek out parks & restaurants near water. _(747 votes, 89.2% agree_) #Human
  +> <Enhancing Greenway Experience>

[Comment 214]: 🙂 More bike trails and walking trails would be a great addition. _(613 votes, 85.8% agree_) #Human
  +> <Enhancing Greenway Experience>

[Comment 146]: 🙂 Our sidewalks need more connectivity. It's too piecemealed and makes some areas unusable. _(510 votes, 87.6% agree_) #Human
  +> <Enhancing Greenway Experience>

[Comment 83]: 🙂 Impact on traffic flow should be considered with any proposed new developments. _(676 votes, 97.0% agree_) #Human
  +> <Improved Traffic Flow>

[Comment 82]: 🙂 Traffic flow needs to be improved throughout Bowling Green, especially on Scottsville Road. _(641 votes, 95.5% agree_) #Human
  +> <Improved Traffic Flow>

[Comment 66]: 🙂 The city should actually time stoplights correctly during rush hour to prevent complete gridlock _(618 votes, 94.8% agree_) #Human
  +> <Improved Traffic Flow>

[Comment 83]: 🙂 Impact on traffic flow should be considered with any proposed new developments. _(676 votes, 97.0% agree_) #Human
  +> <Improved Access and Service Roads>

[Comment 82]: 🙂 Traffic flow needs to be improved throughout Bowling Green, especially on Scottsville Road. _(641 votes, 95.5% agree_) #Human
  +> <Improved Access and Service Roads>

[Comment 66]: 🙂 The city should actually time stoplights correctly during rush hour to prevent complete gridlock _(618 votes, 94.8% agree_) #Human
  +> <Improved Access and Service Roads>

[Comment 83]: 🙂 Impact on traffic flow should be considered with any proposed new developments. _(676 votes, 97.0% agree_) #Human
  +> <Improved Traffic Control Measures>

[Comment 82]: 🙂 Traffic flow needs to be improved throughout Bowling Green, especially on Scottsville Road. _(641 votes, 95.5% agree_) #Human
  +> <Improved Traffic Control Measures>

[Comment 66]: 🙂 The city should actually time stoplights correctly during rush hour to prevent complete gridlock _(618 votes, 94.8% agree_) #Human
  +> <Improved Traffic Control Measures>

[Comment 30]: 🙂 Our local immigrant population has, overall, been a benefit to our city and county. _(923 votes, 70.6% agree_) #Human
  +> <Diversity in City/County Government>

[Comment 34]: 🙂 I would like to see more aid for the homeless whether: tiny house park, health clinic, rehabilitation, job readiness sources. _(728 votes, 79.5% agree_) #Human
  +> <Diversity in City/County Government>

[Comment 3]: 🙂 I should be able to vote in city elections if I pay city taxes, even though I may live in the county. _(760 votes, 75.3% agree_) #Human
  +> <Diversity in City/County Government>

[Comment 12]: 🙂 Medical marijuana should be legal. _(887 votes, 71.5% agree_) #Human
  +> <Voter Support>

[Comment 85]: 🙂 Legalizing marijuana would be a boon for the economy. _(703 votes, 57.8% agree_) #Human
  +> <Voter Support>

[Comment 769]: 🙂 Bowling Green needs to pass a resolution in support of medical cannabis. _(130 votes, 60.0% agree_) #Human
  +> <Voter Support>

[Comment 58]: 🙂 Something needs to be done with the river area- river walk, park, restaurants, summer events- to give the communities ways to get together. _(695 votes, 90.5% agree_) #Human
  +> <Equitable Resource Distribution>

[Comment 60]: 🙂 There isn’t enough here to draw young families. We need a beautification of the riverfront and a more walkable city for young professionals _(746 votes, 80.6% agree_) #Human
  +> <Equitable Resource Distribution>

[Comment 44]: 🙂 I would like to see a sift in city planning and zoning to make BG a more walkable city with better sidewalk planning _(677 votes, 86.1% agree_) #Human
  +> <Equitable Resource Distribution>

[Comment 58]: 🙂 Something needs to be done with the river area- river walk, park, restaurants, summer events- to give the communities ways to get together. _(695 votes, 90.5% agree_) #Human
  +> <Local Business Development and Support>

[Comment 44]: 🙂 I would like to see a sift in city planning and zoning to make BG a more walkable city with better sidewalk planning _(677 votes, 86.1% agree_) #Human
  +> <Local Business Development and Support>

[Comment 18]: 🙂 Beautification projects and other quality-of-life enhancements like parks and libraries help attract new residents and industries to our city/county. _(666 votes, 86.6% agree_) #Human
  +> <Local Business Development and Support>

[Comment 64]: 🙂 More choices when it comes to internet.  BGMU has been offering service to businesses for a while, they should expand to offer to residents. _(631 votes, 96.0% agree_) #Human
  +> <Driving Competition and Improving Services>

[Comment 200]: 🙂 Bowling Green needs more competitive cable rates. _(574 votes, 95.8% agree_) #Human
  +> <Driving Competition and Improving Services>

[Comment 135]: 🙂 Cable companies need competition and not be allowed to raise rates of  customers. Once you sign on, that is your rate unless you upgrade. _(582 votes, 90.7% agree_) #Human
  +> <Driving Competition and Improving Services>

[Comment 64]: 🙂 More choices when it comes to internet.  BGMU has been offering service to businesses for a while, they should expand to offer to residents. _(631 votes, 96.0% agree_) #Human
  +> <Improving Internet Speeds and Competition>

[Comment 200]: 🙂 Bowling Green needs more competitive cable rates. _(574 votes, 95.8% agree_) #Human
  +> <Improving Internet Speeds and Competition>

[Comment 135]: 🙂 Cable companies need competition and not be allowed to raise rates of  customers. Once you sign on, that is your rate unless you upgrade. _(582 votes, 90.7% agree_) #Human
  +> <Improving Internet Speeds and Competition>

[Comment 64]: 🙂 More choices when it comes to internet.  BGMU has been offering service to businesses for a while, they should expand to offer to residents. _(631 votes, 96.0% agree_) #Human
  +> <Driving Competition and Improving Internet Access>

[Comment 200]: 🙂 Bowling Green needs more competitive cable rates. _(574 votes, 95.8% agree_) #Human
  +> <Driving Competition and Improving Internet Access>

[Comment 135]: 🙂 Cable companies need competition and not be allowed to raise rates of  customers. Once you sign on, that is your rate unless you upgrade. _(582 votes, 90.7% agree_) #Human
  +> <Driving Competition and Improving Internet Access>

[Comment 44]: 🙂 I would like to see a sift in city planning and zoning to make BG a more walkable city with better sidewalk planning _(677 votes, 86.1% agree_) #Human
  +> <Historic Preservation and Unique Architectural Heritage>

[Comment 18]: 🙂 Beautification projects and other quality-of-life enhancements like parks and libraries help attract new residents and industries to our city/county. _(666 votes, 86.6% agree_) #Human
  +> <Historic Preservation and Unique Architectural Heritage>

[Comment 182]: 🙂 The arts enrich our city. _(558 votes, 91.0% agree_) #Human
  +> <Historic Preservation and Unique Architectural Heritage>

[Comment 39]: 🙂 All students in Warren county should have the best available education, not the Kentucky minimum, at all times. _(599 votes, 94.2% agree_) #Human
  +> <Comprehensive Sex Education>

[Comment 38]: 🙂 All schools in the county and city should work together to provide the same minimum level of education to students. _(487 votes, 88.3% agree_) #Human
  +> <Comprehensive Sex Education>

[Comment 237]: 🙂 Invest in the youth. _(348 votes, 90.5% agree_) #Human
  +> <Comprehensive Sex Education>

[Comment 57]: 🙂 We need a real bus system. _(670 votes, 80.3% agree_) #Human
  +> <Improved daily commuting>

[Comment 215]: 🙂 A clean, efficient public transportation system would be a huge help to those with limited ability or desire to drive. _(545 votes, 89.2% agree_) #Human
  +> <Improved daily commuting>

[Comment 268]: 🙂 More public transportation options--be creative _(468 votes, 83.1% agree_) #Human
  +> <Improved daily commuting>

[Comment 57]: 🙂 We need a real bus system. _(670 votes, 80.3% agree_) #Human
  +> <Improved Nashville Airport transport>

[Comment 215]: 🙂 A clean, efficient public transportation system would be a huge help to those with limited ability or desire to drive. _(545 votes, 89.2% agree_) #Human
  +> <Improved Nashville Airport transport>

[Comment 268]: 🙂 More public transportation options--be creative _(468 votes, 83.1% agree_) #Human
  +> <Improved Nashville Airport transport>

[Comment 57]: 🙂 We need a real bus system. _(670 votes, 80.3% agree_) #Human
  +> <Real bus system>

[Comment 215]: 🙂 A clean, efficient public transportation system would be a huge help to those with limited ability or desire to drive. _(545 votes, 89.2% agree_) #Human
  +> <Real bus system>

[Comment 268]: 🙂 More public transportation options--be creative _(468 votes, 83.1% agree_) #Human
  +> <Real bus system>

[Comment 24]: 🙂 No family should be homeless in Bowling Green / Warren County, and it is up to our officials to make sure they have options for shelter. _(760 votes, 70.7% agree_) #Human
  +> <Empowering Blue Collar Workers>

[Comment 36]: 🙂 I would like to see a revitalization of older parts of Bowling Green, by giving home improvement grants to home owners. Approval process req _(620 votes, 78.9% agree_) #Human
  +> <Empowering Blue Collar Workers>

[Comment 170]: 🙂 Bowling Green should pass the Uniform Residential Landlord and Tenant Act (or URLTA) to ensure basic standards for landlords and tenants. _(485 votes, 80.0% agree_) #Human
  +> <Empowering Blue Collar Workers>

[Comment 221]: 🙂 Bowling Green's refugee, immigrant, and international student population improves the university and the community. _(758 votes, 69.0% agree_) #Human
  +> <Educated and Cultured Community>

[Comment 15]: 🙂 WKU and local community colleges should continue to offer courses that interest those seeking an education, whether or not the courses aid in job placement. _(641 votes, 78.8% agree_) #Human
  +> <Educated and Cultured Community>

[Comment 56]: 🙂 We need more things to do for ages 1-6. A children’s museum or a small science museum would be wonderful. _(598 votes, 81.3% agree_) #Human
  +> <Educated and Cultured Community>

[Comment 221]: 🙂 Bowling Green's refugee, immigrant, and international student population improves the university and the community. _(758 votes, 69.0% agree_) #Human
  +> <Engaged University-Community Partnership>

[Comment 56]: 🙂 We need more things to do for ages 1-6. A children’s museum or a small science museum would be wonderful. _(598 votes, 81.3% agree_) #Human
  +> <Engaged University-Community Partnership>

[Comment 37]: 🙂 More youth programs are needed to bring the community together: volunteer organizations, community service, innovative creations labs _(531 votes, 78.2% agree_) #Human
  +> <Engaged University-Community Partnership>

[Comment 221]: 🙂 Bowling Green's refugee, immigrant, and international student population improves the university and the community. _(758 votes, 69.0% agree_) #Human
  +> <Culturally Enriched City>

[Comment 70]: 🙂 More concerts should be held in Bowling Green _(535 votes, 86.4% agree_) #Human
  +> <Culturally Enriched City>

[Comment 37]: 🙂 More youth programs are needed to bring the community together: volunteer organizations, community service, innovative creations labs _(531 votes, 78.2% agree_) #Human
  +> <Culturally Enriched City>

[Comment 68]: 🙂 County volunteer fire departments need to be upgraded to fit the current population of Warren County. _(571 votes, 90.9% agree_) #Human
  +> <Modernizing Emergency Services>

[Comment 115]: 🙂 The population has out grown the infrastructure in bg/ warren county. _(454 votes, 83.7% agree_) #Human
  +> <Modernizing Emergency Services>

[Comment 519]: 🙂 County needs to extend water lines in Tuckertown area. About a dozen homes don't have county water or fire hydrants! _(32 votes, 71.9% agree_) #Human
  +> <Modernizing Emergency Services>

[Comment 68]: 🙂 County volunteer fire departments need to be upgraded to fit the current population of Warren County. _(571 votes, 90.9% agree_) #Human
  +> <Enhancing Safety and Quality of Life>

[Comment 99]: 🙂 Warren County should be completely wet, the idea of a “moist” County is outdated and stunting the financial growth and tourism opportunity. _(508 votes, 79.3% agree_) #Human
  +> <Enhancing Safety and Quality of Life>

[Comment 115]: 🙂 The population has out grown the infrastructure in bg/ warren county. _(454 votes, 83.7% agree_) #Human
  +> <Enhancing Safety and Quality of Life>

[Comment 127]: 🙂 City/County officials with connections to a project or proposal should abstain from voting on the project or proposal. _(532 votes, 95.7% agree_) #Human
  +> <Term Limits for Fair Representation>

[Comment 126]: 🙂 City/county officials should disclose to the public via Daily News all connections they have to a project or proposal well before the vote. _(515 votes, 94.4% agree_) #Human
  +> <Term Limits for Fair Representation>

[Comment 61]: 🙂 Individuals should be treated equally. Higher positions or who you work for should not release anyone from responsibility of their actions. _(425 votes, 94.1% agree_) #Human
  +> <Term Limits for Fair Representation>

[Comment 5]: 🙂 Local law enforcement would benefit from additional training in community relations. _(617 votes, 80.6% agree_) #Human
  +> <Police Presence in County Areas>

[Comment 341]: 🙂 People want it all, without paying for anything. A healthy community requires taxes people. _(284 votes, 83.5% agree_) #Human
  +> <Police Presence in County Areas>

[Comment 563]: 🙂 Fireworks use in residential areas is out of control with no police presence to enforce the already in place restrictions. _(115 votes, 73.9% agree_) #Human
  +> <Police Presence in County Areas>

[Comment 104]: 🙂 Downtown redevelopment should retain accessible parking for existing businesses. _(528 votes, 93.9% agree_) #Human
  +> <Parking Regulation Enforcement>

[Comment 202]: 🙂 Residential Parking Permits for areas near WKU. We can rescue our yards from parking. We pay property taxes for the students' convenience, _(230 votes, 77.8% agree_) #Human
  +> <Parking Regulation Enforcement>

[Comment 348]: 🙂 Extra police, or parking enforcement unit should be on duty during events to maintain legal parking _(222 votes, 69.4% agree_) #Human
  +> <Parking Regulation Enforcement>

[Comment 104]: 🙂 Downtown redevelopment should retain accessible parking for existing businesses. _(528 votes, 93.9% agree_) #Human
  +> <Downtown Parking Improvement>

[Comment 392]: 🙂 More parking downtown.  There are too many reserved spaces which should be available in the evening. _(314 votes, 85.0% agree_) #Human
  +> <Downtown Parking Improvement>

[Comment 348]: 🙂 Extra police, or parking enforcement unit should be on duty during events to maintain legal parking _(222 votes, 69.4% agree_) #Human
  +> <Downtown Parking Improvement>

[Comment 104]: 🙂 Downtown redevelopment should retain accessible parking for existing businesses. _(528 votes, 93.9% agree_) #Human
  +> <Parking Program Improvement>

[Comment 191]: 🙂 Increase green spaces, especially near mall, to decrease congestion and improve "green" status _(486 votes, 78.2% agree_) #Human
  +> <Parking Program Improvement>

[Comment 202]: 🙂 Residential Parking Permits for areas near WKU. We can rescue our yards from parking. We pay property taxes for the students' convenience, _(230 votes, 77.8% agree_) #Human
  +> <Parking Program Improvement>

[Comment 36]: 🙂 I would like to see a revitalization of older parts of Bowling Green, by giving home improvement grants to home owners. Approval process req _(620 votes, 78.9% agree_) #Human
  +> <Enhancing Aesthetic Appeal>

[Comment 366]: 🙂 Too many trees have been torn down to make way for new developments. We should re-plant more trees around Bowling Green. _(412 votes, 92.0% agree_) #Human
  +> <Enhancing Aesthetic Appeal>

[Comment 371]: 🙂 Bowling Green is an excellent place to raise a family, shop, worship, get an education, and recreate. I feel safe in most places here in BG. _(330 votes, 93.0% agree_) #Human
  +> <Enhancing Aesthetic Appeal>

[Comment 142]: 🙂 There is not enough done for tourism in Warren Co. We offer much more than Corvettes and we should be promoting other activities. _(575 votes, 82.1% agree_) #Human
  +> <Boosting Tourism and Economy>

[Comment 99]: 🙂 Warren County should be completely wet, the idea of a “moist” County is outdated and stunting the financial growth and tourism opportunity. _(508 votes, 79.3% agree_) #Human
  +> <Boosting Tourism and Economy>

[Comment 320]: 🙂 The Barren River water front needs to be improved on both sides of the river and include  canoeing and kayaking water features in the river. _(385 votes, 92.7% agree_) #Human
  +> <Boosting Tourism and Economy>

[Comment 93]: 🙂 Industrial hemp should be re-legalized to grow in Kentucky. _(565 votes, 79.6% agree_) #Human
  +> <Economic Growth>

[Comment 85]: 🙂 Legalizing marijuana would be a boon for the economy. _(703 votes, 57.8% agree_) #Human
  +> <Economic Growth>

[Comment 129]: 🙂 Recreational Cannabis in KY offers tourism, increased revenue, job opportunities, and shows massive leadership in the southern region. _(563 votes, 56.5% agree_) #Human
  +> <Economic Growth>

[Comment 302]: 🙂 Recycling services need to be expanded & improved. With growing population, think of how much land will become dump sites...Ridiculous! _(488 votes, 88.1% agree_) #Human
  +> <Responsible Businesses>

[Comment 308]: 🙂 Work needs to be done on improving litter pick up and encouraging people dispose of their litter correctly. _(333 votes, 89.8% agree_) #Human
  +> <Responsible Businesses>

[Comment 208]: 🙂 BG has a bad problem with littering.  My husband and I fill up to 5 recycle bins every week between downtown and campus, cleaning it all up. _(327 votes, 89.6% agree_) #Human
  +> <Responsible Businesses>

[Comment 170]: 🙂 Bowling Green should pass the Uniform Residential Landlord and Tenant Act (or URLTA) to ensure basic standards for landlords and tenants. _(485 votes, 80.0% agree_) #Human
  +> <Enforcing Noise Ordinances>

[Comment 371]: 🙂 Bowling Green is an excellent place to raise a family, shop, worship, get an education, and recreate. I feel safe in most places here in BG. _(330 votes, 93.0% agree_) #Human
  +> <Enforcing Noise Ordinances>

[Comment 174]: 🙂 Bowling Green needs stricter code enforcement on property use and condition in some neighborhoods. _(360 votes, 78.3% agree_) #Human
  +> <Enforcing Noise Ordinances>

[Comment 237]: 🙂 Invest in the youth. _(348 votes, 90.5% agree_) #Human
  +> <Physical Activity for Health>

[Comment 236]: 🙂 Actually teach HS students useful things. _(245 votes, 92.7% agree_) #Human
  +> <Physical Activity for Health>

[Comment 421]: 🙂 High sugar items like pop tarts and chocolate muffins should not be a part of the public school breakfast menu _(219 votes, 69.9% agree_) #Human
  +> <Physical Activity for Health>

[Comment 308]: 🙂 Work needs to be done on improving litter pick up and encouraging people dispose of their litter correctly. _(333 votes, 89.8% agree_) #Human
  +> <Deter Littering>

[Comment 208]: 🙂 BG has a bad problem with littering.  My husband and I fill up to 5 recycle bins every week between downtown and campus, cleaning it all up. _(327 votes, 89.6% agree_) #Human
  +> <Deter Littering>

[Comment 414]: 🙂 Mandatory, expensive fines should be implemented for littering and illegal dumping. _(299 votes, 95.3% agree_) #Human
  +> <Deter Littering>

[Comment 308]: 🙂 Work needs to be done on improving litter pick up and encouraging people dispose of their litter correctly. _(333 votes, 89.8% agree_) #Human
  +> <Clean Yards>

[Comment 208]: 🙂 BG has a bad problem with littering.  My husband and I fill up to 5 recycle bins every week between downtown and campus, cleaning it all up. _(327 votes, 89.6% agree_) #Human
  +> <Clean Yards>

[Comment 414]: 🙂 Mandatory, expensive fines should be implemented for littering and illegal dumping. _(299 votes, 95.3% agree_) #Human
  +> <Clean Yards>

[Comment 382]: 🙂 Smoking should not be allowed in public parks. _(273 votes, 70.7% agree_) #Human
  +> <Healthier Environment>

[Comment 381]: 🙂 Smoking should not be allowed within 25 feet of a business. _(235 votes, 68.5% agree_) #Human
  +> <Healthier Environment>

```


## Argument Map for Entire Dataset

In [5]:
# output discussion title and question
output = argdown_heading(summary.topic, summary.description)

# argumentTopicSupport is a view that joins arguments and topics, and adds support and agreeability scores

# get unique topicId from argumentTopicSupport
# output each topic once
for topicId in argumentTopicSupport.df.get_column('topicId').unique().sort().to_list():
    topic = topics.get(topicId)
    heading = topic['Heading']
    title = topic['Title']
    output += argdown_topic(heading, title)

# # topic hierarchy
# for Parent_ID, Parent_Name, topicList, Child_Left_ID, Child_Left_Name, Child_Right_ID, Child_Right_Name, Distance in hierarchicalTopics.df.iter_rows():
#     output += "\n"
#     output += f"[Topic {Parent_ID}]: Node #AI \n"
#     output += f"  + [Topic {Child_Left_ID}]\n"
#     output += f"  + [Topic {Child_Right_ID}]\n"

# output all arguments and link to topic
for topicId, topicHeading, argumentId, argumentTitle, argumentContent, agreeability, agrees, disagrees in (
    argumentTopicSupport.df
    .select('topicId', 'topicHeading', 'argumentId', 'argumentTitle', 'argumentContent', 'agreeability', 'agrees', 'disagrees')
        .iter_rows()):

    body = f"{argumentContent} ({agrees + disagrees} voters approximated, {agreeability*100:.1f}% tend to agree)"
    output += argdown_argument(argumentTitle, body, topicHeading)


argumentCommentMapSupport = argumentCommentMap.df.filter(
    relationship='SUPPORT')

# output all unique comments present in the argumentCommentMap
for commentId, commentText, agrees, disagrees, agreeability in (
    argumentCommentMapSupport
    .select('commentId').unique()
    .join(comments.df, on='commentId', how='left')
    .select('commentId', 'commentText', 'agrees', 'disagrees', 'agreeability')
    .iter_rows()
):
    body = f"{commentText} ({agrees + disagrees} votes, {agreeability*100:.1f}% agree)"
    output += argdown_comment(f'Comment {commentId}', body)

# enumerate argument-comment relationships
for argumentId, topicId, argumentTitle, commentIds in (
    argumentCommentMapSupport.lazy()
    # add argumentTitle
    .join(arguments.df.lazy(), on=['topicId', 'argumentId'], how='left')
    .select('argumentId', 'topicId', 'argumentTitle', 'commentId')
    # aggregate commentIds
    .group_by(['argumentId', 'topicId', 'argumentTitle']).all()
).collect().iter_rows():
    output += f"\n<{argumentTitle}>\n"
    for commentId in commentIds:
        output += argdown_supported_by(f'Comment {commentId}')

display_markdown(argdown_markdown_template(output), raw=True)

```argdown-map
===
sourceHighlighter:
    removeFrontMatter: true
webComponent:
    withoutMaximize: true
    height: 640px
model:
    removeTagsFromText: true
# map:
#     statementLabelMode: text
#     argumentLabelMode: text
selection:
    excludeDisconnected: false
color:
    tagColors:
        Human: "#99cc99"
        AI: "#9999cc"
===


# Improving Bowling Green / Warren County

What do you believe should change in Bowling Green/Warren County in order to make it a better place to live, work and spend time?

[Addressing Homelessness with Occupational Taxes]: 🤖 Implementing Occupational Taxes to Address Homelessness #AI

[Improving Traffic Flow]: 🤖 Implementation of Traffic Control Measures to Improve Road Flow #AI

[No to Fairness Ordinance]: 🤖 Opposition to the Proposed Fairness Ordinance in Bowling Green #AI

[College Campus Improvements]: 🤖 Implementing a Comprehensive Parking and Food Assistance Program for College Students on Campus #AI

[Improving City Planning and Zoning]: 🤖 Implementing a Comprehensive City Planning and Zoning Strategy for Better Neighborhoods and Community Activities #AI

[Opioid Epidemic and Healthcare]: 🤖 The Role of Doctors and Teenagers in the Opioid Crisis #AI

[Public School Funding and Accountability]: 🤖 The Importance of Adequate Funding and Accountability in Public School Education #AI

[Community Enrichment Hub]: 🤖 Bowling Green Community Enrichment Center: A Multi-Purpose Hub for Sports, Arts, and Learning #AI

[Pedestrian-Friendly Infrastructure]: 🤖 Implementation of Pedestrian-Friendly Infrastructure in Key Areas #AI

[Enhancing Community Safety and Tourism]: 🤖 Implementing Water and Fire Protection Measures in Warren County's Tuckertown Area #AI

[Internet Choices and Competition]: 🤖 Expansion of Residential Internet Services in Bowling Green #AI

[Marijuana Legalization]: 🤖 Legalized Marijuana: A Revenue-Generating Solution for Bowling Green #AI

[Enhancing Recycling and Waste Management]: 🤖 Implementing Improved Recycling and Waste Management Services in Bowling Green #AI

[Improved Transit Services]: 🤖 Enhanced Public Transportation System for Accessible Daily Commute #AI

<Driving Competition and Improving Services>: 🤖 Increasing competition for cable and internet companies will drive down rates, improve services, and provide more fiber options for Bowling Green residents. (1193 voters approximated, 94.6% tend to agree) #AI
  +> [Internet Choices and Competition]

<Improving Internet Speeds and Competition>: 🤖 Making fiber internet options available will improve internet speeds, drive down rates, and promote competition for cable and internet companies in Bowling Green. (1184 voters approximated, 94.6% tend to agree) #AI
  +> [Internet Choices and Competition]

<Driving Competition and Improving Internet Access>: 🤖 The expansion of residential internet services, including the provision of fiber options, will promote competition, drive down rates, and improve internet speeds and accessibility for Bowling Green residents. (1188 voters approximated, 94.5% tend to agree) #AI
  +> [Internet Choices and Competition]

<Ensuring Fair Cable TV Services>: 🤖 Implementing regulations on cable TV providers, such as including all local channels and controlling rates, will ensure fair pricing and better services for Bowling Green residents. (968 voters approximated, 94.2% tend to agree) #AI
  +> [Internet Choices and Competition]

<Bridging the Digital Divide>: 🤖 Promoting internet infrastructure growth and expanding services to underserved areas will improve internet accessibility and bridge the digital divide in Bowling Green. (1034 voters approximated, 93.9% tend to agree) #AI
  +> [Internet Choices and Competition]

<Bridging the Digital Divide through Free WiFi>: 🤖 Providing free WiFi in certain areas will improve internet accessibility and bridge the digital divide in Bowling Green. (1015 voters approximated, 93.9% tend to agree) #AI
  +> [Internet Choices and Competition]

<Improved Traffic Flow>: 🤖 Reducing the number of traffic lights and implementing roundabouts can significantly improve traffic flow, ensuring a smoother driving experience for drivers. (1273 voters approximated, 92.9% tend to agree) #AI
  +> [Improving Traffic Flow]

<Improved Access and Service Roads>: 🤖 Proper planning and improvement of service roads and access roads can significantly improve safety and convenience for drivers, reducing traffic jams and ensuring a smoother driving experience. (1299 voters approximated, 92.7% tend to agree) #AI
  +> [Improving Traffic Flow]

<Comprehensive Sex Education>: 🤖 Comprehensive sexual education is necessary to lower STD and teen pregnancy rates. (907 voters approximated, 92.2% tend to agree) #AI
  +> [Public School Funding and Accountability]

<Improved Traffic Control Measures>: 🤖 Implementing traffic control measures such as roundabouts, traffic lights, and turn lanes can significantly improve road flow and safety, reducing traffic congestion and ensuring a smoother driving experience. (1325 voters approximated, 92.1% tend to agree) #AI
  +> [Improving Traffic Flow]

<Safer and More Convenient Roads>: 🤖 Widening roads and creating shoulders can significantly improve safety and convenience for drivers, providing more space for turning lanes and ensuring a smoother driving experience. (1288 voters approximated, 92.0% tend to agree) #AI
  +> [Improving Traffic Flow]

<Improved Traffic Enforcement>: 🤖 Better enforcement of traffic violations can significantly improve safety and convenience for drivers, reducing the risk of accidents and ensuring a smoother driving experience. (1287 voters approximated, 92.0% tend to agree) #AI
  +> [Improving Traffic Flow]

<Improved Traffic Flow and Safety>: 🤖 The installation of roundabouts and properly timed traffic lights can significantly improve safety and convenience for drivers, ensuring smooth traffic flow and reducing the risk of accidents. (1326 voters approximated, 91.8% tend to agree) #AI
  +> [Improving Traffic Flow]

<Improved Road Aesthetics>: 🤖 Implementing underground utilities and improving road aesthetics can significantly enhance the driving experience, ensuring a more pleasing and visually appealing environment for drivers. (1059 voters approximated, 91.6% tend to agree) #AI
  +> [Improving Traffic Flow]

<Improved Peak Hour Traffic Flow>: 🤖 Improving traffic flow during peak hours can significantly reduce traffic congestion, ensuring a smoother driving experience for drivers. (1313 voters approximated, 91.4% tend to agree) #AI
  +> [Improving Traffic Flow]

<Deter Littering>: 🤖 Implementing fines can deter littering and illegal dumping, promoting a cleaner and safer environment. (690 voters approximated, 91.2% tend to agree) #AI
  +> [Enhancing Recycling and Waste Management]

<Clean Yards>: 🤖 Enforcing cleanliness and reducing junk cars can significantly improve the appearance of Bowling Green. (690 voters approximated, 91.2% tend to agree) #AI
  +> [Enhancing Recycling and Waste Management]

<Responsible Businesses>: 🤖 Requiring businesses to take responsibility for litter can significantly reduce waste and promote a cleaner environment. (796 voters approximated, 90.6% tend to agree) #AI
  +> [Enhancing Recycling and Waste Management]

<Term Limits for Fair Representation>: 🤖 Implement term limits for local elected officials to promote fair representation, prevent misuse of power, and encourage fresh perspectives. (951 voters approximated, 90.5% tend to agree) #AI
  +> [Addressing Homelessness with Occupational Taxes]

<Modern Recycling>: 🤖 Modernizing recycling services can enhance efficiency, reduce waste, and improve overall satisfaction. (797 voters approximated, 90.3% tend to agree) #AI
  +> [Enhancing Recycling and Waste Management]

<Accessible Recycling>: 🤖 Increasing recycling and trash bins can help decrease litter and promote a cleaner environment. (797 voters approximated, 90.3% tend to agree) #AI
  +> [Enhancing Recycling and Waste Management]

<Improved Driver Education and Enforcement>: 🤖 Improving driver education and enforcement of traffic laws can significantly reduce dangerous driving, ensuring a safer and more convenient driving experience for drivers. (1199 voters approximated, 90.3% tend to agree) #AI
  +> [Improving Traffic Flow]

<Modernizing Emergency Services>: 🤖 Upgrading volunteer fire departments will better equip them to serve the growing population and ensure the safety of Warren County residents. (694 voters approximated, 90.2% tend to agree) #AI
  +> [Enhancing Community Safety and Tourism]

<Sustainable Recycling>: 🤖 Requiring recycling services for businesses and multi-family dwellings can significantly reduce waste and promote sustainability. (760 voters approximated, 90.0% tend to agree) #AI
  +> [Enhancing Recycling and Waste Management]

<Physical Activity for Health>: 🤖 Daily physical activity in schools is vital to combating childhood obesity and promoting healthy lifestyles. (529 voters approximated, 90.0% tend to agree) #AI
  +> [Public School Funding and Accountability]

<Historic Preservation and Unique Architectural Heritage>: 🤖 Preserving historic homes and buildings, and maintaining unique architectural heritage are essential for fostering a strong sense of community, celebrating Bowling Green's rich history, and enhancing the city's overall appeal. (1209 voters approximated, 89.4% tend to agree) #AI
  +> [Improving City Planning and Zoning]

<Essential Education Funding>: 🤖 Increased funding for public schools, especially art education programs, is essential to fostering a well-rounded education for students. (1217 voters approximated, 89.2% tend to agree) #AI
  +> [Public School Funding and Accountability]

<Enhancing Safety and Quality of Life>: 🤖 Extending water lines and implementing fire protection in the Tuckertown area will significantly improve safety and quality of life for residents, while reducing insurance costs. (808 voters approximated, 89.2% tend to agree) #AI
  +> [Enhancing Community Safety and Tourism]

<Consistent Education Standards>: 🤖 Merging city and county school systems is necessary to ensure consistent education standards. (1121 voters approximated, 89.1% tend to agree) #AI
  +> [Public School Funding and Accountability]

<Youth Investment>: 🤖 Investing in the youth through education and job opportunities is crucial for the future workforce. (1300 voters approximated, 89.0% tend to agree) #AI
  +> [Public School Funding and Accountability]

<Equitable Resource Distribution>: 🤖 Ensuring equitable distribution of resources across all areas of the city is vital for fostering a more inclusive and thriving community, where every resident has access to essential amenities and opportunities. (1368 voters approximated, 88.9% tend to agree) #AI
  +> [Improving City Planning and Zoning]

<Local Business Development and Support>: 🤖 Encouraging local business development and supporting small businesses is essential for Bowling Green's unique character, as it fosters a stronger sense of community and boosts the local economy. (1335 voters approximated, 88.7% tend to agree) #AI
  +> [Improving City Planning and Zoning]

<Enhancing Aesthetic Appeal>: 🤖 Enforcing zoning mandates will improve the aesthetic appeal of Bowling Green and maintain property values. (936 voters approximated, 88.6% tend to agree) #AI
  +> [No to Fairness Ordinance]

<Green Spaces and Shaded Areas>: 🤖 Providing more green spaces and shaded areas will significantly enhance the quality of life for Bowling Green residents, promoting healthier lifestyles and fostering a stronger sense of community. (1247 voters approximated, 88.5% tend to agree) #AI
  +> [Improving City Planning and Zoning]

<Comprehensive City Planning and Zoning Strategy>: 🤖 To make Bowling Green a better place to live, work, and spend time, it's crucial to implement a comprehensive city planning and zoning strategy that enforces historic preservation codes, supports local businesses, provides family-centered activities, allocates resources equitably, and focuses on mixed-use development and renovations. (1424 voters approximated, 87.9% tend to agree) #AI
  +> [Improving City Planning and Zoning]

<Boosting Tourism and Economy>: 🤖 Implementing a marketing campaign and promoting diverse tourism activities will attract more visitors, boosting the local economy and enhancing Warren County's appeal. (817 voters approximated, 87.9% tend to agree) #AI
  +> [Enhancing Community Safety and Tourism]

<Existing Housing Renovations and Improvements>: 🤖 Focusing on the existing housing in BG, including renovations and improvements, will preserve the city's character, foster a stronger sense of community, and promote sustainable growth. (1184 voters approximated, 87.0% tend to agree) #AI
  +> [Improving City Planning and Zoning]

<Improving Downtown Pedestrian Experience>: 🤖 Making downtown a pedestrian-friendly mall can significantly enhance the overall pedestrian experience and promote local businesses. (1204 voters approximated, 86.5% tend to agree) #AI
  +> [Pedestrian-Friendly Infrastructure]

<Walkability and Mixed-Use Developments>: 🤖 Increasing walkability and promoting mixed-use developments will create a more vibrant, sustainable, and livable community, where residents can easily access essential services and amenities. (1378 voters approximated, 86.4% tend to agree) #AI
  +> [Improving City Planning and Zoning]

<Family-Centered Activities and Entertainment>: 🤖 Offering a wide range of family-centered activities and entertainment options will significantly improve the quality of life for Bowling Green residents, making it a more attractive place to live, work, and spend time. (1332 voters approximated, 86.3% tend to agree) #AI
  +> [Improving City Planning and Zoning]

<Improving Pedestrian Safety>: 🤖 Installing sidewalks in areas with heavy pedestrian traffic can significantly improve pedestrian safety and accessibility. (1213 voters approximated, 86.2% tend to agree) #AI
  +> [Pedestrian-Friendly Infrastructure]

<Parking Regulation Enforcement>: 🤖 Monitoring and enforcing parking regulations and handicap spaces will ensure fair access to parking and promote safety within the community. (718 voters approximated, 85.9% tend to agree) #AI
  +> [College Campus Improvements]

<Enhancing Greenway Experience>: 🤖 A continuous and connected Greenway trail system can significantly enhance the overall pedestrian and recreational experience. (1261 voters approximated, 85.9% tend to agree) #AI
  +> [Pedestrian-Friendly Infrastructure]

<Enhancing Waterfront Experience>: 🤖 Improving river access and developing areas near water can significantly enhance the overall recreational and tourism experience. (1192 voters approximated, 85.8% tend to agree) #AI
  +> [Pedestrian-Friendly Infrastructure]

<Improved daily commuting>: 🤖 Expanding the public transportation system's service area, increasing accessibility, and adding more stops in residential areas will greatly improve daily commuting for those without cars. (1064 voters approximated, 85.6% tend to agree) #AI
  +> [Improved Transit Services]

<Improved Nashville Airport transport>: 🤖 Providing regular and affordable daily transport to Nashville Airport will greatly benefit both residents and visitors. (1064 voters approximated, 85.6% tend to agree) #AI
  +> [Improved Transit Services]

<Enhancing Park Experience>: 🤖 Implementing pedestrian-friendly infrastructure in sports-centric parks can enhance the overall park experience and promote healthier lifestyle choices. (1262 voters approximated, 85.6% tend to agree) #AI
  +> [Pedestrian-Friendly Infrastructure]

<Educated and Cultured Community>: 🤖 Increasing the number of cultural and educational establishments would promote learning, personal growth, and community development. (1285 voters approximated, 85.5% tend to agree) #AI
  +> [Community Enrichment Hub]

<Downtown Parking Improvement>: 🤖 Improving downtown parking and accessibility by increasing parking spaces, making reserved spaces available during evenings, and enforcing parking regulations will benefit existing businesses and their customers. (762 voters approximated, 85.4% tend to agree) #AI
  +> [College Campus Improvements]

<Real bus system>: 🤖 Implementing a real bus system will significantly improve public transportation in the area. (1037 voters approximated, 85.3% tend to agree) #AI
  +> [Improved Transit Services]

<Expanding Access to Resources>: 🤖 Providing a public library branch in the southern end of Warren County will improve access to resources and educational opportunities for residents in that area. (503 voters approximated, 85.3% tend to agree) #AI
  +> [Enhancing Community Safety and Tourism]

<Enhanced BG buses>: 🤖 Expanding the service area and improving accessibility of existing BG buses will greatly benefit commuters in Bowling Green. (1034 voters approximated, 85.1% tend to agree) #AI
  +> [Improved Transit Services]

<Bridging the Digital Divide>: 🤖 Ensuring high-speed internet access for all residents will support economic growth, remote work opportunities, and improved quality of life. (649 voters approximated, 84.9% tend to agree) #AI
  +> [Enhancing Community Safety and Tourism]

<Equitable School Redistricting>: 🤖 Redistricting city schools is essential to ensure fairness and equality in education. (1019 voters approximated, 84.9% tend to agree) #AI
  +> [Public School Funding and Accountability]

<Expanded Nashville transportation>: 🤖 Increasing public transportation options and service to Nashville, including a regular and affordable daily transport to Nashville Airport, will benefit both residents and visitors. (1069 voters approximated, 84.8% tend to agree) #AI
  +> [Improved Transit Services]

<Enhanced transportation accessibility>: 🤖 Improving transportation accessibility for seniors and disabled individuals will enable their independence and mobility. (1069 voters approximated, 84.8% tend to agree) #AI
  +> [Improved Transit Services]

<Diversity in City/County Government>: 🤖 Promote racial and ethnic diversity in city/county government to enhance representation, recruitment, and growth. (1329 voters approximated, 84.8% tend to agree) #AI
  +> [Addressing Homelessness with Occupational Taxes]

<Engaged University-Community Partnership>: 🤖 Fostering cooperation between WKU and Bowling Green would enhance learning experiences, promote community engagement, and prepare students for their future careers. (1141 voters approximated, 84.4% tend to agree) #AI
  +> [Community Enrichment Hub]

<Culturally Enriched City>: 🤖 Engaging art installations would enrich Bowling Green's cultural landscape, fostering creativity and community pride. (1156 voters approximated, 84.3% tend to agree) #AI
  +> [Community Enrichment Hub]

<Dietary Accommodation>: 🤖 Providing special diet options in schools is essential to accommodate students with cultural differences or dietary needs. (773 voters approximated, 84.2% tend to agree) #AI
  +> [Public School Funding and Accountability]

<Police Presence in County Areas>: 🤖 Increase police presence and resources in county areas to ensure public safety and address domestic violence disputes effectively. (731 voters approximated, 84.1% tend to agree) #AI
  +> [Addressing Homelessness with Occupational Taxes]

<Grocery Store Competition>: 🤖 Introduce new grocery stores to increase competition, lower prices, and improve quality for consumers. (570 voters approximated, 84.0% tend to agree) #AI
  +> [Addressing Homelessness with Occupational Taxes]

<Parking Program Improvement>: 🤖 Implementing a comprehensive parking program with garages, green spaces, and connected lots near WKU campus will significantly improve parking accessibility and enforcement for college students. (852 voters approximated, 83.7% tend to agree) #AI
  +> [College Campus Improvements]

<Charter School Accountability>: 🤖 Holding charter schools accountable is crucial to prevent financial strain on public school funding. (980 voters approximated, 83.7% tend to agree) #AI
  +> [Public School Funding and Accountability]

<Improve medical care access>: 🤖 Ensuring timely access to medical care, including appointments and healthcare facilities, is crucial to alleviate suffering and prevent health conditions from worsening. (937 voters approximated, 83.4% tend to agree) #AI
  +> [Opioid Epidemic and Healthcare]

<Improving Pedestrian Safety on Major Roads>: 🤖 Building sidewalks along major roads can significantly improve pedestrian safety and accessibility. (1092 voters approximated, 83.2% tend to agree) #AI
  +> [Pedestrian-Friendly Infrastructure]

<Enforcing Noise Ordinances>: 🤖 Enforcing noise ordinances and codes will maintain peace and order in residential areas, ensuring a more enjoyable living experience for all Bowling Green residents. (802 voters approximated, 82.8% tend to agree) #AI
  +> [No to Fairness Ordinance]

<Inclusive Arts & Culture>: 🤖 Promoting diversity in concert and performing arts activities would celebrate Bowling Green's multicultural community, fostering inclusivity and unity. (1118 voters approximated, 82.7% tend to agree) #AI
  +> [Community Enrichment Hub]

<Empowering Mentorship>: 🤖 Increasing the number of accessible mentorship programs would empower young adults and teenagers by providing them with guidance, support, and role models. (1178 voters approximated, 82.5% tend to agree) #AI
  +> [Community Enrichment Hub]

<Improving Cycling Experience>: 🤖 Implementing more bike trails and wider bike lanes can significantly improve the safety and accessibility for cyclists. (1112 voters approximated, 82.3% tend to agree) #AI
  +> [Pedestrian-Friendly Infrastructure]

<Immigrant and Refugee Integration>: 🤖 Integrating immigrants and refugees into the community is essential for fostering a more diverse, inclusive, and vibrant society, where every resident has the opportunity to thrive. (1110 voters approximated, 82.3% tend to agree) #AI
  +> [Improving City Planning and Zoning]

<Empowering Blue Collar Workers>: 🤖 Investing in the North-Northeast ends of town will empower blue collar workers and contribute to a more equitable Bowling Green community. (995 voters approximated, 82.2% tend to agree) #AI
  +> [No to Fairness Ordinance]

<Community Policing Improvement>: 🤖 Enhancing community policing and neighborhood patrols by having cops live and work in the same areas will improve safety and trust within the community. (127 voters approximated, 81.9% tend to agree) #AI
  +> [College Campus Improvements]

<Tenant Support in Downtown District>: 🤖 Offer resources, training, and support for downtown tenants to enhance their business skills, promote success, and reduce turnover. (1110 voters approximated, 81.7% tend to agree) #AI
  +> [Addressing Homelessness with Occupational Taxes]

<Strong Landlord-Tenant Laws>: 🤖 Establish stronger tenant and renter rights, enforce leash laws, and encourage property maintenance to preserve neighborhoods and ensure quality housing. (1272 voters approximated, 81.5% tend to agree) #AI
  +> [Addressing Homelessness with Occupational Taxes]

<Rental Assistance and Affordability>: 🤖 Introduce rent control laws, create quality affordable rental units, and offer rental assistance programs to address high rental prices and limited options. (1207 voters approximated, 81.5% tend to agree) #AI
  +> [Addressing Homelessness with Occupational Taxes]

<Implement stricter opioid regulations>: 🤖 Enacting stricter regulations for opioid drugs and expanding mental health treatment access will help mitigate the opioid crisis and its devastating consequences. (1053 voters approximated, 81.2% tend to agree) #AI
  +> [Opioid Epidemic and Healthcare]

<Enforcing Noise Ordinances and Supporting Mental Health>: 🤖 Enforcing noise ordinances and improving mental health awareness will contribute to a more peaceful and supportive Bowling Green community. (1032 voters approximated, 80.9% tend to agree) #AI
  +> [No to Fairness Ordinance]

<Enhanced Youth Development>: 🤖 Establishing more after-school activity centers and extending their operating hours would provide youth with additional opportunities for personal growth, learning, and socialization. (1149 voters approximated, 80.6% tend to agree) #AI
  +> [Community Enrichment Hub]

<Diversifying Food Options>: 🤖 Providing diverse and healthy food options will cater to various dietary needs and contribute to a healthier Bowling Green community. (919 voters approximated, 80.4% tend to agree) #AI
  +> [No to Fairness Ordinance]

<Affordable Housing and Property Ownership>: 🤖 Increase affordable housing options and enforce stronger landlord/tenant laws to preserve neighborhoods and promote responsible property ownership. (1327 voters approximated, 80.3% tend to agree) #AI
  +> [Addressing Homelessness with Occupational Taxes]

<Poverty Reduction and Elderly Support>: 🤖 Establish affordable support programs and job training initiatives to reduce poverty and support the elderly and low-income residents. (1307 voters approximated, 80.0% tend to agree) #AI
  +> [Addressing Homelessness with Occupational Taxes]

<Educate about drug risks>: 🤖 Educating students about drug risks and side effects through drug information classes and school curricula will empower them to make informed decisions and reduce drug use. (846 voters approximated, 79.7% tend to agree) #AI
  +> [Opioid Epidemic and Healthcare]

<Fostering Workforce Development>: 🤖 Enhancing workforce development will help fill job openings, contributing to economic growth and prosperity in Warren County. (137 voters approximated, 79.6% tend to agree) #AI
  +> [Enhancing Community Safety and Tourism]

<Occupational Tax for Homeless Support>: 🤖 Implement a small occupational tax on businesses to fund temporary housing and support services for the homeless, creating a safer and more prosperous community for all. (1176 voters approximated, 78.8% tend to agree) #AI
  +> [Addressing Homelessness with Occupational Taxes]

<Enhancing Scenic Beauty>: 🤖 Outlawing billboards will preserve the natural beauty and enhance the visual appeal of Bowling Green. (1068 voters approximated, 77.7% tend to agree) #AI
  +> [No to Fairness Ordinance]

<Modernizing Urban Development>: 🤖 Implementing Greenville SC-style development will modernize Bowling Green's urban landscape, promote walkability, and attract young talent. (1035 voters approximated, 76.1% tend to agree) #AI
  +> [No to Fairness Ordinance]

<Active Community>: 🤖 Enhancing indoor sports facilities would encourage physical activity, promote teamwork, and cater to diverse athletic interests. (977 voters approximated, 75.5% tend to agree) #AI
  +> [Community Enrichment Hub]

<Enhance addiction treatment and rehabilitation>: 🤖 Implementing comprehensive probation rehab programs that offer addiction treatment, life and job skills training, and housing assistance will help break the cycle of addiction and recidivism. (1127 voters approximated, 75.4% tend to agree) #AI
  +> [Opioid Epidemic and Healthcare]

<Address drug addiction as a health issue>: 🤖 Treating drug addiction as a health problem and offering rehabilitation services instead of imprisonment for drug possession arrestees will help reduce recidivism and effectively address the root cause of drug use. (1127 voters approximated, 75.3% tend to agree) #AI
  +> [Opioid Epidemic and Healthcare]

<Healthier Community>: 🤖 Expanding indoor swimming pool availability would encourage healthier lifestyles, promote water safety, and provide therapeutic options for residents. (545 voters approximated, 74.9% tend to agree) #AI
  +> [Community Enrichment Hub]

<Green Path and Parklike Spaces>: 🤖 Encouraging walkable green paths and shaded parklike spaces in apartment buildings will improve tenants' quality of life and promote sustainability. (590 voters approximated, 74.6% tend to agree) #AI
  +> [College Campus Improvements]

<Regulate over-prescribing doctors>: 🤖 To effectively combat the opioid crisis, doctors who over-prescribe opioids should face harsher penalties, and medical societies must actively regulate their members to prevent drug dependency. (1098 voters approximated, 73.5% tend to agree) #AI
  +> [Opioid Epidemic and Healthcare]

<Healthier Environment>: 🤖 Implement policies to prohibit smoking in public parks and within 25 feet of businesses to address the concerns of 70% and 68% of voters, respectively, promoting a healthier environment. (327 voters approximated, 71.3% tend to agree) #AI
  +> [Marijuana Legalization]

<Food Access Improvement>: 🤖 Improving food options and access near WKU campus by building a grocery store and providing food assistance programs will significantly benefit college students' health and well-being. (740 voters approximated, 70.9% tend to agree) #AI
  +> [College Campus Improvements]

<Sign and Green Space Regulation>: 🤖 Regulating business signs and promoting green spaces will improve the aesthetic and environmental appeal of Bowling Green/Warren County. (776 voters approximated, 70.9% tend to agree) #AI
  +> [College Campus Improvements]

<Fostering Inclusivity and Economic Growth>: 🤖 Enforcing a fairness ordinance and attracting diverse businesses will foster inclusivity and promote economic growth in Bowling Green. (1335 voters approximated, 68.6% tend to agree) #AI
  +> [No to Fairness Ordinance]

<Voter Support>: 🤖 Pass a resolution in support of medical cannabis to address the needs of 60% of voters and enhance the quality of life in Bowling Green. (1000 voters approximated, 65.8% tend to agree) #AI
  +> [Marijuana Legalization]

<Food Assistance Program>: 🤖 Establishing a food assistance program for college students will help address food insecurity and support their overall well-being. (140 voters approximated, 64.3% tend to agree) #AI
  +> [College Campus Improvements]

<Economic Growth>: 🤖 Re-legalize industrial hemp in Kentucky to address the needs of 79% of voters, promoting economic growth and job opportunities. (909 voters approximated, 63.4% tend to agree) #AI
  +> [Marijuana Legalization]

<Economic Benefits>: 🤖 Legalize recreational marijuana to address the needs of 57% of voters, creating substantial economic benefits and job opportunities for Bowling Green. (1179 voters approximated, 59.8% tend to agree) #AI
  +> [Marijuana Legalization]

<Revenue Generation>: 🤖 Implement a substantial tax on marijuana sales to address the needs of 57% of voters, generating significant revenue for Bowling Green. (1094 voters approximated, 56.2% tend to agree) #AI
  +> [Marijuana Legalization]

[Comment 283]: 🙂 Driving in this town very dangerous, Drivers Education classes should be required for all new drivers!!! (338 votes, 43.2% agree) #Human

[Comment 545]: 🙂 Higher education is intended to help young people develop into informed, well-rounded citizens who contribute positively to society. (148 votes, 79.7% agree) #Human

[Comment 286]: 🙂 The entire bypass area is an eyesore and there needs to be a concerted effort like a TIF to revitalize this crucial part of town. (458 votes, 59.8% agree) #Human

[Comment 149]: 🙂 Why isn't Nashville Rd between the roundabout and Campbell Ln wider? Getting stuck there at 5 o'clock is a plague. (384 votes, 77.6% agree) #Human

[Comment 155]: 🙂 We need to promote competition in town for Charter. AT&T isn't much better and doesn't serve everywhere. Need more fiber to the homes. (499 votes, 88.8% agree) #Human

[Comment 292]: 🙂 Spectrum should have competition and not a monopoly. Rates should be regulated by city county government (504 votes, 79.4% agree) #Human

[Comment 24]: 🙂 No family should be homeless in Bowling Green / Warren County, and it is up to our officials to make sure they have options for shelter. (760 votes, 41.3% agree) #Human

[Comment 542]: 🙂 Indoor swimming pools for public use and therapy year around without time restrictions due to special swim leagues. Encourage healthier life (142 votes, 66.2% agree) #Human

[Comment 3]: 🙂 I should be able to vote in city elections if I pay city taxes, even though I may live in the county. (760 votes, 50.5% agree) #Human

[Comment 557]: 🙂 Make Scott Waste follow their contract - Return trash cans next to the house and not leave them on the sidewalks.  Keep BG clean! (116 votes, 58.6% agree) #Human

[Comment 158]: 🙂 SKYPAC needs to bring in younger, more entertaining acts. It will fall apart if they keep only catering to 50+. Could be a great venue. (377 votes, 67.1% agree) #Human

[Comment 688]: 🙂 The face of drug dealers has changed. Today most opioid drug dealers are doctors or teenagers in the suburbs ordering from China. (39 votes, 23.1% agree) #Human

[Comment 12]: 🙂 Medical marijuana should be legal. (887 votes, 43.0% agree) #Human

[Comment 679]: 🙂 State and federal medical societies are unable or unwilling to police their members regarding the drug dependency problem. (29 votes, 17.2% agree) #Human

[Comment 819]: 🙂 City officials need to spread resources across city and not just focus on wealthy neighborhoods. Low-income areas of city are ignored. (76 votes, 52.6% agree) #Human

[Comment 798]: 🙂 The traffic intersection at Scottsville Rd and Pascoe Blvd needs some immediate attention as far as congestion.  It's very unsafe. (47 votes, 61.7% agree) #Human

[Comment 414]: 🙂 Mandatory, expensive fines should be implemented for littering and illegal dumping. (299 votes, 90.6% agree) #Human

[Comment 423]: 🙂 BGMU needs to offer residential fiber internet as a UTILITY, using Chattanooga as a perfect example. (219 votes, 83.6% agree) #Human

[Comment 667]: 🙂 Bowling Green needs more access to affordable , healthy, local food sources. (115 votes, 63.5% agree) #Human

[Comment 18]: 🙂 Beautification projects and other quality-of-life enhancements like parks and libraries help attract new residents and industries to our city/county. (666 votes, 73.3% agree) #Human

[Comment 816]: 🙂 Businesses and multi-family dwellings should be required to offer and use recycling services. (128 votes, 65.6% agree) #Human

[Comment 551]: 🙂 We  a true junior college which offers credit / non credit courses like many other states from basket weaving to cyber security (45 votes, 2.2% agree) #Human

[Comment 405]: 🙂 Allow residence of an area to vote on new developments that might have impact on them opposed to commissioners/magistrates getting kickbacks (268 votes, 68.7% agree) #Human

[Comment 536]: 🙂 Recycling should be encouraged by not charging for the service (206 votes, 50.5% agree) #Human

[Comment 804]: 🙂 Access Rd, east side of Scottsville Rd. in shopping dist causes congestion. Give right of way to access road or cross streets, not a mix! (43 votes, 48.8% agree) #Human

[Comment 268]: 🙂 More public transportation options--be creative (468 votes, 66.2% agree) #Human

[Comment 411]: 🙂 Should be an interstate exit for the Woodburn area it could help decrease BG congestion and let people bypass BG all together (170 votes, 45.9% agree) #Human

[Comment 807]: 🙂 More walking/nature trails in our sports-centric parks. Have certain hours where walkers can enjoy the beauty of Paul Walker Golf Course. (100 votes, 44.0% agree) #Human

[Comment 554]: 🙂 Water safety and swim lessons to all k-12 students. (118 votes, 30.5% agree) #Human

[Comment 21]: 🙂 The arts are an important component of K-12 education. (787 votes, 79.9% agree) #Human

[Comment 548]: 🙂 We need a 6 day a week farmer's market that is open at least 5 hours - (118 votes, 32.2% agree) #Human

[Comment 30]: 🙂 Our local immigrant population has, overall, been a benefit to our city and county. (923 votes, 41.3% agree) #Human

[Comment 673]: 🙂 BG needs to increase recycling and trash bins around the parks and parkways to decrease trash and litter (164 votes, 80.5% agree) #Human

[Comment 533]: 🙂 Every new apartment building should have shaded parklike space for refreshment of its tenants (106 votes, 1.9% agree) #Human

[Comment 801]: 🙂 Other counties have school at home on snow days so that kids do not have to make up days.  This should be considered for here. (96 votes, 54.2% agree) #Human

[Comment 15]: 🙂 WKU and local community colleges should continue to offer courses that interest those seeking an education, whether or not the courses aid in job placement. (641 votes, 57.6% agree) #Human

[Comment 265]: 🙂 Improved businesses such as banks and groceries in lowest income areas (324 votes, 59.3% agree) #Human

[Comment 146]: 🙂 Our sidewalks need more connectivity. It's too piecemealed and makes some areas unusable. (510 votes, 75.3% agree) #Human

[Comment 140]: 🙂 Big names and corporations get away with too much. We need fairer regulation and level playing fields for the small and big guys alike. (321 votes, 53.9% agree) #Human

[Comment 179]: 🙂 Zoning changes should only be made with the consent of a majority of the neighbors in the affected area. (522 votes, 55.6% agree) #Human

[Comment 572]: 🙂 City Annexed areas need more city services for their tax $$. (97 votes, 73.2% agree) #Human

[Comment 441]: 🙂 We need mixed-use development in the most disadvantaged areas of BG. (163 votes, 64.4% agree) #Human

[Comment 581]: 🙂 there should be a tenant/landlord grievance department with legal assistance (141 votes, 56.0% agree) #Human

[Comment 33]: 🙂 I would like to see part of the downtown area converted to an "art district". Something similar to the Wynnwood Walls in Miami, FL. (343 votes, 3.2% agree) #Human

[Comment 170]: 🙂 Bowling Green should pass the Uniform Residential Landlord and Tenant Act (or URLTA) to ensure basic standards for landlords and tenants. (485 votes, 60.0% agree) #Human

[Comment 715]: 🙂 Free WIFI in certain areas - particularly around Fountain Squre (138 votes, 68.1% agree) #Human

[Comment 563]: 🙂 Fireworks use in residential areas is out of control with no police presence to enforce the already in place restrictions. (115 votes, 47.8% agree) #Human

[Comment 295]: 🙂 Our city should make a bid for Google Fiber to help compete with Spectrum and AT&T. (436 votes, 83.5% agree) #Human

[Comment 57]: 🙂 We need a real bus system. (670 votes, 60.6% agree) #Human

[Comment 709]: 🙂 Drug information classes should be given to incoming freshmen of WKU to educate them about the risks and health hazards. (97 votes, 25.8% agree) #Human

[Comment 438]: 🙂 Term limits should be initiated on county executives. (249 votes, 66.3% agree) #Human

[Comment 301]: 🙂 Planning & Zoning need to do a better job of listening & working with residents. It's not next to their house, so they don't care! (333 votes, 67.6% agree) #Human

[Comment 63]: 🙂 WKU should upgrade their CIT program to keep up with the times (97 votes, 44.3% agree) #Human

[Comment 587]: 🙂 BGDN should publish each month the number of job positions filled / people employed through the Workforce Development Board’s efforts. (98 votes, 30.6% agree) #Human

[Comment 846]: 🙂 Solving the drug crisis should not mean people with health problems/injuries have to suffer with severe pain. (60 votes, 83.3% agree) #Human

[Comment 444]: 🙂 It should not take 3 months to get a gynecologist appointment in this city. (95 votes, 68.4% agree) #Human

[Comment 185]: 🙂 Refugee neighbors enrich our city's landscape and we should find more ways to welcome and integrate them into the life of the city. (749 votes, 30.6% agree) #Human

[Comment 578]: 🙂 apartment complexes should be required to participate in recycling program (234 votes, 76.1% agree) #Human

[Comment 322]: 🙂 WKU should consider building a parking structure at the Chestnut St lot. (129 votes, 11.6% agree) #Human

[Comment 718]: 🙂 A way to improve Bowling Green and the surrounding areas is to improve traffic flow & circulation by limiting the number of traffic lights. (76 votes, 57.9% agree) #Human

[Comment 316]: 🙂 Go to underground utilities. Scottsville rd and the by-pass look awful. (355 votes, 71.8% agree) #Human

[Comment 39]: 🙂 All students in Warren county should have the best available education, not the Kentucky minimum, at all times. (599 votes, 88.3% agree) #Human

[Comment 700]: 🙂 Resources to better integrate immigrants and refugees into the community should be put into place. (106 votes, 32.1% agree) #Human

[Comment 191]: 🙂 Increase green spaces, especially near mall, to decrease congestion and improve "green" status (486 votes, 56.4% agree) #Human

[Comment 834]: 🙂 Support non-public (or public charter) school choice options, and make choice viable for families of ALL income levels. (89 votes, 1.1% agree) #Human

[Comment 703]: 🙂 All public schools should have an adequate summer feeding program so no child goes hungry. (163 votes, 58.3% agree) #Human

[Comment 60]: 🙂 There isn’t enough here to draw young families. We need a beautification of the riverfront and a more walkable city for young professionals (746 votes, 61.1% agree) #Human

[Comment 697]: 🙂 Greek Life should be monitored on WKU's campus. (111 votes, 64.0% agree) #Human

[Comment 822]: 🙂 Fines and other enforcement actions should be taken against landlords who don't keep up with their properties. (136 votes, 86.8% agree) #Human

[Comment 182]: 🙂 The arts enrich our city. (558 votes, 82.1% agree) #Human

[Comment 54]: 🙂 Temp svc need to furnish the equivalent of FMLA, despite not having 50 people at one single site. (152 votes, 17.1% agree) #Human

[Comment 453]: 🙂 Plan for people, not cars. Walking should be enjoyable, not a hazard. (246 votes, 66.7% agree) #Human

[Comment 450]: 🙂 Build parking areas. (87 votes, 5.7% agree) #Human

[Comment 435]: 🙂 many parts of the By-pass can and should be widened (261 votes, 78.5% agree) #Human

[Comment 429]: 🙂 Apartments for 55 and older that are upscale and lovely. (145 votes, 55.9% agree) #Human

[Comment 584]: 🙂 The city & county school districts need to be re-evaluated The yards/houses that touch Potter Grays playground r all County school districts (83 votes, 59.0% agree) #Human

[Comment 706]: 🙂 There needs to be better access to grocery stores and restaurants from WKU's campus. (109 votes, 33.9% agree) #Human

[Comment 36]: 🙂 I would like to see a revitalization of older parts of Bowling Green, by giving home improvement grants to home owners. Approval process req (620 votes, 57.7% agree) #Human

[Comment 176]: 🙂 Enact rent control laws to stabilize the constant increase in rates. (435 votes, 15.9% agree) #Human

[Comment 352]: 🙂 Houston "courtesy police" maintain safety on access rds enforcing right of way & parking/litter rules, assist traffic flow - we need! (104 votes, 19.2% agree) #Human

[Comment 486]: 🙂 install traffic cameras and enforce running of stop lights. more traffic enforcement means more $$$ for the city. (154 votes, 19.5% agree) #Human

[Comment 468]: 🙂 Widen Cemetry Road, Russellville Road, Lover’s Lane, Campbell Lane, The Old ByPass, and Scotsville Road from old ByPass to Natcher Parkway. (93 votes, 16.1% agree) #Human

[Comment 87]: 🙂 A fairness ordinance in Bowling Green would attract new businesses. No one wants to set up show in a town ethically stuck in the 1800s. (918 votes, 10.7% agree) #Human

[Comment 69]: 🙂 Something needs to be done to improve traffic flow (606 votes, 90.8% agree) #Human

[Comment 72]: 🙂 We need more activities for families, such as a large-scale all-in-one kids play arena like All About Kids. (386 votes, 30.6% agree) #Human

[Comment 200]: 🙂 Bowling Green needs more competitive cable rates. (574 votes, 91.6% agree) #Human

[Comment 590]: 🙂 Local companies that can't fill job openings should recruit from parts of KY w/ high unemployment & pay relocation expenses. (79 votes, 39.2% agree) #Human

[Comment 742]: 🙂 There should be more effort put into teaching students about any potential harmful side effects of drugs, illegal or otherwise. (87 votes, 65.5% agree) #Human

[Comment 66]: 🙂 The city should actually time stoplights correctly during rush hour to prevent complete gridlock (618 votes, 89.6% agree) #Human

[Comment 751]: 🙂 Sex education should include information about consent, safe sex practices, and STD spread and prevention. (119 votes, 69.7% agree) #Human

[Comment 730]: 🙂 Sewer should be required to be run to all new subdivisions and cost should be split between city/county water systems and builders (75 votes, 78.7% agree) #Human

[Comment 870]: 🙂 Provide accessible transportation for seniors that have difficulty driving (130 votes, 86.2% agree) #Human

[Comment 340]: 🙂 Some county roads, such as Hammett Hill Road, are so narrow it's hard for two cars to pass. Surely they could make them a little wider. (167 votes, 56.9% agree) #Human

[Comment 882]: 🙂 Create shoulders on the bypass. (52 votes, 30.8% agree) #Human

[Comment 218]: 🙂 Developing more bike trails into surrounding areas of the city would allow for a more developed both transport and recreational experiences (524 votes, 67.2% agree) #Human

[Comment 93]: 🙂 Industrial hemp should be re-legalized to grow in Kentucky. (565 votes, 59.3% agree) #Human

[Comment 736]: 🙂 More creative learning classes (54 votes, 37.0% agree) #Human

[Comment 861]: 🙂 Hawaii outlaws billboards tokeep the landscape from being blocked I would love to see Bowling Green billboard free (57 votes, 29.8% agree) #Human

[Comment 212]: 🙂 Leash laws should be more strictly enforced in both the city and county (418 votes, 47.4% agree) #Human

[Comment 483]: 🙂 people need to understand that a STOP sign means STOP, not just slow down and keep going. (157 votes, 78.3% agree) #Human

[Comment 739]: 🙂 There should be a push to improve mental health awareness and availability in Bowling Green. (96 votes, 79.2% agree) #Human

[Comment 227]: 🙂 We need to block left turns on the busy parts of the by-pass so people don't block everyone else trying to turn. (312 votes, 23.1% agree) #Human

[Comment 221]: 🙂 Bowling Green's refugee, immigrant, and international student population improves the university and the community. (758 votes, 38.0% agree) #Human

[Comment 459]: 🙂 Lease laws should be enacted and enforced in the county as they are in the city. (115 votes, 54.8% agree) #Human

[Comment 724]: 🙂 How is it that surrounding counties have county wide high speed internet, yet one third of Warren County residents have slow or no internet? (80 votes, 85.0% agree) #Human

[Comment 206]: 🙂 Better ordinances on keeping animals on leashes or in fenced areas are needed. (327 votes, 47.4% agree) #Human

[Comment 203]: 🙂 Sidewalks are needed along Gary Farms Blvd (279 votes, 41.2% agree) #Human

[Comment 748]: 🙂 There should be more cooperation between WKU and Bowling Green, especially for things like internships and community involvement. (148 votes, 86.5% agree) #Human

[Comment 608]: 🙂 Bowling Green needs more cultural and educational establishments, like sciemce, history and art museums. (190 votes, 50.5% agree) #Human

[Comment 215]: 🙂 A clean, efficient public transportation system would be a huge help to those with limited ability or desire to drive. (545 votes, 78.3% agree) #Human

[Comment 209]: 🙂 Traffic improvement on the so-called Bypass.  Most cities actually have a bypass.  Bowling Green doesn't have one. (389 votes, 66.6% agree) #Human

[Comment 596]: 🙂 Expand Nashville Rd all the way past Chaney’s & Buchanan park.  Very dangerous area.. very much so for a school area & young drivers. (101 votes, 62.4% agree) #Human

[Comment 513]: 🙂 Please increase more family friendly activities (160 votes, 73.8% agree) #Human

[Comment 260]: 🙂 Rental assistance programs (267 votes, 17.6% agree) #Human

[Comment 129]: 🙂 Recreational Cannabis in KY offers tourism, increased revenue, job opportunities, and shows massive leadership in the southern region. (563 votes, 13.0% agree) #Human

[Comment 769]: 🙂 Bowling Green needs to pass a resolution in support of medical cannabis. (130 votes, 20.0% agree) #Human

[Comment 126]: 🙂 City/county officials should disclose to the public via Daily News all connections they have to a project or proposal well before the vote. (515 votes, 88.7% agree) #Human

[Comment 498]: 🙂 Require 4 yr HS lifeskills class, how to: family finance, file tax, repair/maintain car, repair/maintain home, cook/sew, apply/interview job (196 votes, 68.4% agree) #Human

[Comment 382]: 🙂 Smoking should not be allowed in public parks. (273 votes, 41.4% agree) #Human

[Comment 891]: 🙂 After school activity centers for youth with later hours of operation. (68 votes, 73.5% agree) #Human

[Comment 248]: 🙂 Shopping should be spread across all four corners of the county, so the traffic on Scottsville rd and Campbell lane will be improved. (271 votes, 42.4% agree) #Human

[Comment 99]: 🙂 Warren County should be completely wet, the idea of a “moist” County is outdated and stunting the financial growth and tourism opportunity. (508 votes, 58.7% agree) #Human

[Comment 784]: 🙂 Have a Dual Slalom Course, Pump Track & Bike Skills Course co-located by the Low Hollow Mountain Bike Trail at Weldon Peete Park. (37 votes, 2.7% agree) #Human

[Comment 245]: 🙂 With jails crowded, it seems many are first time offenders of crimes that hurt no one, nor property. Seems rehab would be better. (395 votes, 55.4% agree) #Human

[Comment 364]: 🙂 Junking up highways/roadways with signs should not be the primary way tourism is promoted. (252 votes, 72.2% agree) #Human

[Comment 111]: 🙂 City and county parks departments should provide more programming for adults and seniors. (411 votes, 66.9% agree) #Human

[Comment 236]: 🙂 Actually teach HS students useful things. (245 votes, 85.3% agree) #Human

[Comment 885]: 🙂 The main problem facing is the workforce development. There is over 6500 jobs open in Warren County. This is a big issue. (55 votes, 67.3% agree) #Human

[Comment 114]: 🙂 Minimum wage required for businesses to receive tax credits should be raised to a living wage (681 votes, 31.0% agree) #Human

[Comment 653]: 🙂 More citizens need to be involved in the future development of BG. Too many developers who benefit financially are calling the shots. (147 votes, 94.6% agree) #Human

[Comment 894]: 🙂 Doctors should not overreact to the opioid crisis. People in pain shouldn't suffer because others lack self control and abuse the system. (43 votes, 25.6% agree) #Human

[Comment 775]: 🙂 BG needs to redevelop from WKU to the river and beyond. Model after Greenville SC (94 votes, 59.6% agree) #Human

[Comment 379]: 🙂 The city of Bowling Green should enforce their existing noise ordinance. (238 votes, 81.5% agree) #Human

[Comment 766]: 🙂 A kayak trail from Lampkin Park to Barren River. Improved River access. (106 votes, 71.7% agree) #Human

[Comment 251]: 🙂 Make it a walking city. Sidewalks along the main avenues (scottsville rd, campbell ln, 31 bypass) with plants, benches, art pieces. (540 votes, 59.6% agree) #Human

[Comment 641]: 🙂 Need a better way to work with tenants in the downtown district. Too many come and go because they lack business skills. (69 votes, 24.6% agree) #Human

[Comment 522]: 🙂 No business should get tax credit without paying  workers at least $10 per hour plus benefits that would increase that to minimum of $15. (143 votes, 23.1% agree) #Human

[Comment 635]: 🙂 Concerts and performing arts activities must reflect the diverse racial/ethnic demographic of the city/county. (140 votes, 30.0% agree) #Human

[Comment 650]: 🙂 Do SOMETHING about completing the "wrap" downtown, or at least enhance (and that includes restaurant service) what is currently there. (111 votes, 83.8% agree) #Human

[Comment 519]: 🙂 County needs to extend water lines in Tuckertown area. About a dozen homes don't have county water or fire hydrants! (32 votes, 43.8% agree) #Human

[Comment 888]: 🙂 Better visibility on roads. Reflectors and better signage (54 votes, 70.4% agree) #Human

[Comment 686]: 🙂 We know fentanyl is coming through our post office. Law enforcement should have dogs finding it before it kills the next youth. (48 votes, 41.7% agree) #Human

[Comment 19]: 🙂 More nonviolent inmates convicted of drug possession should be set free. (768 votes, 15.6% agree) #Human

[Comment 668]: 🙂 There should be officers more readily available to the county areas; ex. for domestic violence disputes. (74 votes, 67.6% agree) #Human

[Comment 534]: 🙂 Every citizen should be within 1 mile of some shaded green space. (81 votes, 18.5% agree) #Human

[Comment 147]: 🙂 The homeless population is getting out of hand. We need to find better methods for curtailing and rehabing and keeping them off the streets. (360 votes, 68.3% agree) #Human

[Comment 16]: 🙂 The main purpose of higher education is job training. (779 votes, 4.5% agree) #Human

[Comment 409]: 🙂 The rental market prices are well beyond an affordable housing level and needs to be addressed given the number of renters in BG. (190 votes, 47.4% agree) #Human

[Comment 796]: 🙂 We need a significantly sized public library branch in the southern end of Warren County near South Warren. (64 votes, 50.0% agree) #Human

[Comment 159]: 🙂 We need to commit more resources to the North-Northeast ends of town. Blue collar people who help make Bowling Green what is. Too neglected. (258 votes, 61.2% agree) #Human

[Comment 805]: 🙂 Scottsville Rd should be 4+ lanes between I-65 & Lovers Lane, get rid of the service Rd. (58 votes, 41.4% agree) #Human

[Comment 415]: 🙂 The intersection of Scottsville road and entrance to Greenview Hospital needs to be widened (117 votes, 26.5% agree) #Human

[Comment 421]: 🙂 High sugar items like pop tarts and chocolate muffins should not be a part of the public school breakfast menu (219 votes, 39.7% agree) #Human

[Comment 683]: 🙂 Doctors over-prescribing opioids should be sentenced at least as harshly as street dealers if not more harshly due to their knowledge. (113 votes, 55.8% agree) #Human

[Comment 4]: 🙂 Most local officials run for office to serve the public good. (282 votes, 21.3% agree) #Human

[Comment 144]: 🙂 We should be incentivizing higher paying jobs coming to BG. Cost of living is outrunning wages and it's starting to be a problem. (497 votes, 73.4% agree) #Human

[Comment 394]: 🙂 Five lane Cemetery Road from Interchange to By Pass, except four lanes through the Cemetery with median prohibiting left turns. (141 votes, 36.2% agree) #Human

[Comment 531]: 🙂 Better enforcement of traffic violations. Too many running red lights, parking in fire lanes, cell phone usage while driving (211 votes, 72.5% agree) #Human

[Comment 150]: 🙂 We need look for a Natcher connection on Three Springs or Elrod Rd (246 votes, 65.9% agree) #Human

[Comment 275]: 🙂 We need urban planning to ensure that mixed use housing, shopping and new neighborhoods are created to make our community better (428 votes, 78.5% agree) #Human

[Comment 528]: 🙂 All new and existing single family housing should have at least one shade tree planting (108 votes, 31.5% agree) #Human

[Comment 659]: 🙂 For a city of this size, there is too little available in the way of weekend activities for Seniors who are not ready for rocking chairs. (106 votes, 58.5% agree) #Human

[Comment 799]: 🙂 Community Education provides after-school care necessary to working parents.  Funding should not be cut. (135 votes, 82.2% agree) #Human

[Comment 266]: 🙂 Job training programs for lower income (396 votes, 79.3% agree) #Human

[Comment 293]: 🙂 There are intersections such as the access road along Scottsville that to be reworked, possibly into roundabout or similar designs. (256 votes, 57.8% agree) #Human

[Comment 546]: 🙂 Connect business parking lots to reduce unnecessary traffic on main roadways (125 votes, 61.6% agree) #Human

[Comment 10]: 🙂 In order to better combat the opioid epidemic, it is time to view drug addiction as a health problem rather than a criminal justice problem. (839 votes, 59.0% agree) #Human

[Comment 13]: 🙂 Recreational marijuana should be legal. (953 votes, 9.1% agree) #Human

[Comment 135]: 🙂 Cable companies need competition and not be allowed to raise rates of  customers. Once you sign on, that is your rate unless you upgrade. (582 votes, 81.4% agree) #Human

[Comment 156]: 🙂 The intersection at Taco Bell and KFC is a nightmare (471 votes, 47.8% agree) #Human

[Comment 287]: 🙂 parks and rec should promote and support pickleball for older citizens (88 votes, 20.5% agree) #Human

[Comment 153]: 🙂 The Old Morgantown Rd train crossing needs either an underpass or an overpass (195 votes, 21.0% agree) #Human

[Comment 790]: 🙂 Many people want something for nothing and if they don't get it they steal it instead of working for it.  We are producing a gimme society. (83 votes, 8.4% agree) #Human

[Comment 400]: 🙂 Bowling Green needs an indoor / outdoor sports event complex/ (224 votes, 8.0% agree) #Human

[Comment 263]: 🙂 Post-secondary training programs for special education students. (342 votes, 71.9% agree) #Human

[Comment 662]: 🙂 Bowling Green should work on becoming a more food sustainable city with less food waste and food insecurity (95 votes, 55.8% agree) #Human

[Comment 671]: 🙂 Bowling Green should be become a carbon neutral city. (68 votes, 26.5% agree) #Human

[Comment 138]: 🙂 Publicizing more cultural activities other than the Bowling Green International Festival. (445 votes, 67.2% agree) #Human

[Comment 412]: 🙂 A senior living community, designed for 50+ residents rather than 75+ & in failing health residents, is needed — and apparently ignored. (141 votes, 56.0% agree) #Human

[Comment 269]: 🙂 Encourage connection between the wealthy and the poor (234 votes, 50.4% agree) #Human

[Comment 850]: 🙂 We must ensure historic preservation codes are properly enforced in our historic districts; lack of consistency erodes our unique aesthetic. (114 votes, 78.9% agree) #Human

[Comment 558]: 🙂 Owners of large parking lots should be required to break them up with pockets of green space and trees; perhaps create a walkable green path (159 votes, 13.2% agree) #Human

[Comment 588]: 🙂 Local companies that can't fill job openings & the Workforce Dev't. Board should set up commuter lots and a shuttle service to rural areas. (79 votes, 57.0% agree) #Human

[Comment 177]: 🙂 People in Bowling Green should not be discriminated against based on their sexual orientation. (617 votes, 75.7% agree) #Human

[Comment 37]: 🙂 More youth programs are needed to bring the community together: volunteer organizations, community service, innovative creations labs (531 votes, 56.3% agree) #Human

[Comment 692]: 🙂 Bowling Green should have more healthy/clean eating options for those who have limited diets. (97 votes, 38.1% agree) #Human

[Comment 561]: 🙂 Parking  on both sides of the streets  in apartment subdivisions should  should not be allowed  and police need to enforce the law. (64 votes, 34.4% agree) #Human

[Comment 689]: 🙂 stronger enforcement of junk cars in yards and trashy looking yards in general (110 votes, 58.2% agree) #Human

[Comment 320]: 🙂 The Barren River water front needs to be improved on both sides of the river and include  canoeing and kayaking water features in the river. (385 votes, 85.5% agree) #Human

[Comment 55]: 🙂 Every school has 1-9% homeless people. This is for the government to deal with via economic policy.  Not sending to a NGO or giving a shanty (201 votes, 5.5% agree) #Human

[Comment 171]: 🙂 Bowling Green should spend less time redoing the same sidewalks over and over again and focus on the areas that actually need improvement. (302 votes, 71.5% agree) #Human

[Comment 61]: 🙂 Individuals should be treated equally. Higher positions or who you work for should not release anyone from responsibility of their actions. (425 votes, 88.2% agree) #Human

[Comment 820]: 🙂 Locally owned, small businesses should get incentives and assistance -- not chains and outside investors. (82 votes, 70.7% agree) #Human

[Comment 847]: 🙂 Build parking garage behind Spencer’s, remove some parking from Ft. Square. Add a green space on top of garage! Soccer field or rooftop food (75 votes, 22.7% agree) #Human

[Comment 713]: 🙂 Better parking around the square would draw in more college students who like to shop but can not find parking downtown. (121 votes, 42.1% agree) #Human

[Comment 174]: 🙂 Bowling Green needs stricter code enforcement on property use and condition in some neighborhoods. (360 votes, 56.7% agree) #Human

[Comment 189]: 🙂 Encourage local business development with grants, etc. For a city this size, it is ridiculous that we have basically one local coffee shop! (543 votes, 42.2% agree) #Human

[Comment 168]: 🙂 When resurfacing a roadway, attention should be given to making the manhole covers level with the new surface. (436 votes, 90.8% agree) #Human

[Comment 183]: 🙂 We should focus downtown development dollars on rehabilitation and reuse of buildings that can be done so, rather than building new. (544 votes, 78.3% agree) #Human

[Comment 832]: 🙂 There are not enough family-centered activities to do down town. (92 votes, 60.9% agree) #Human

[Comment 180]: 🙂 Bowling Green needs Community Enrichment Classes that include woodworking, welding, gardening and general home beautification for hobbyist. (498 votes, 71.1% agree) #Human

[Comment 299]: 🙂 The money I earn & spend is all done within the city limits but as a county resident I have NO voice/vote in city politics. Needs to change! (451 votes, 40.6% agree) #Human

[Comment 192]: 🙂 Require new downtown developments to become LEED certified (178 votes, 33.7% agree) #Human

[Comment 585]: 🙂 BGDN should publish these BGPD stats weekly – 1) traffic citations 2) vehicle accidents reported; 3) traffic accidents resulting in injury. (101 votes, 16.8% agree) #Human

[Comment 567]: 🙂 Whenever land is bought by the city/county, the seller and amount sold for needs to be transparent, rather than “LLC” as listed seller. (142 votes, 70.4% agree) #Human

[Comment 43]: 🙂 I would like to see the WKU psych and Medical programs collaborate on mental health research, which is so sadly lacking in today's society. (411 votes, 54.7% agree) #Human

[Comment 302]: 🙂 Recycling services need to be expanded & improved. With growing population, think of how much land will become dump sites...Ridiculous! (488 votes, 76.2% agree) #Human

[Comment 844]: 🙂 Support more local chefs and brewers! White Squirrel is fantastic and I want to see more of it in BG! (65 votes, 69.2% agree) #Human

[Comment 311]: 🙂 There should be more officers observing intersections. (214 votes, 26.2% agree) #Human

[Comment 34]: 🙂 I would like to see more aid for the homeless whether: tiny house park, health clinic, rehabilitation, job readiness sources. (728 votes, 59.1% agree) #Human

[Comment 308]: 🙂 Work needs to be done on improving litter pick up and encouraging people dispose of their litter correctly. (333 votes, 79.6% agree) #Human

[Comment 579]: 🙂 we need transit buses for elderly and disabled in the county and they should be easy to enter and exit/wheelchair accessible (153 votes, 62.1% agree) #Human

[Comment 695]: 🙂 Bowling Green should have basic zoning mandates in order to beautify decrepit buildings/gravel lawns/general trashiness. (105 votes, 63.8% agree) #Human

[Comment 701]: 🙂 Public school systems need to provide more funding for art education programs, such as music, theater, and visual arts. (118 votes, 50.8% agree) #Human

[Comment 436]: 🙂 Reducing poverty will enhance the quality of life for everyone in our community regardless of status. (232 votes, 72.4% agree) #Human

[Comment 710]: 🙂 More local produce should be incorporated into public school's lunchrooms (145 votes, 72.4% agree) #Human

[Comment 64]: 🙂 More choices when it comes to internet.  BGMU has been offering service to businesses for a while, they should expand to offer to residents. (631 votes, 92.1% agree) #Human

[Comment 58]: 🙂 Something needs to be done with the river area- river walk, park, restaurants, summer events- to give the communities ways to get together. (695 votes, 81.0% agree) #Human

[Comment 430]: 🙂 Need new schools in Warren County to keep up with population growth, CTE is falling down, Drakes and Greenwood high are way overcrowded (82 votes, 46.3% agree) #Human

[Comment 582]: 🙂 fines for leaving dog feces in public areas needs to be enforced (159 votes, 64.8% agree) #Human

[Comment 564]: 🙂 We should have a commuting service between here & Nashville. (155 votes, 61.3% agree) #Human

[Comment 314]: 🙂 BG can link/support hotel/food, recreational/cultural venues in a Tourism Passport, giving discounts when at least 3 are purchased together. (256 votes, 53.1% agree) #Human

[Comment 433]: 🙂 There should be more opportunities for the arts and sciences in the Housing Authority so the students there can rise above! (122 votes, 47.5% agree) #Human

[Comment 326]: 🙂 Some of the roads in the city are not wide enough to put sidewalks in but they do anyway catherine dr is ridiculous (75 votes, 20.0% agree) #Human

[Comment 484]: 🙂 Either do the speed limit or get over to the right, to many SLOW, out of county drivers clogging up the roadways. (108 votes, 20.4% agree) #Human

[Comment 204]: 🙂 Why has the Police stop enforcing parking regulations? Park any way any direction. Is it time for a separate Traffic Enforcement Division? (150 votes, 18.7% agree) #Human

[Comment 743]: 🙂 City schools should be redistricted to more evenly distribute the kids. (54 votes, 59.3% agree) #Human

[Comment 97]: 🙂 The lack of leadership regarding adequate planning and zoning in BG is discouraging. (341 votes, 67.2% agree) #Human

[Comment 85]: 🙂 Legalizing marijuana would be a boon for the economy. (703 votes, 15.5% agree) #Human

[Comment 752]: 🙂 Promote access to mental health treatment, rather than treating addicts as criminals. (104 votes, 65.4% agree) #Human

[Comment 356]: 🙂 I wish we had train service to Nashville (342 votes, 62.6% agree) #Human

[Comment 207]: 🙂 Relax restrictions on fencing in Home Owners Associations to promote people keeping animals on leashes or in fenced yards. (162 votes, 48.1% agree) #Human

[Comment 228]: 🙂 We need to grow the internet infrastructure particularly in growing areas of the county like Alvaton. (443 votes, 88.3% agree) #Human

[Comment 749]: 🙂 We need more Community Farming Programs(where you pay farmers money up front and during harvest you get fresh fruits/veggies, eggs, etc) (91 votes, 38.5% agree) #Human

[Comment 70]: 🙂 More concerts should be held in Bowling Green (535 votes, 72.7% agree) #Human

[Comment 213]: 🙂 We need to make it worthwhile for builders to create more quality affordable rental units for residents, rather than more luxury units. (437 votes, 47.8% agree) #Human

[Comment 603]: 🙂 Louisville rd area needs more businesses and restaurants extending out towards the factories to also bring in revenue from Edmonson Co. (137 votes, 65.0% agree) #Human

[Comment 82]: 🙂 Traffic flow needs to be improved throughout Bowling Green, especially on Scottsville Road. (641 votes, 91.0% agree) #Human

[Comment 874]: 🙂 Cable TV providers should include all local channels. (66 votes, 84.8% agree) #Human

[Comment 737]: 🙂 The area of downtown and the riverfront should continue to be developed for entertainment and recreation for both day and night activities. (147 votes, 95.9% agree) #Human

[Comment 868]: 🙂 Provide regular and affordable daily transport to Nashville Airport from Bowling Green. (79 votes, 44.3% agree) #Human

[Comment 731]: 🙂 We need to renovate the run down businesses and store fronts, instead of building new places (152 votes, 69.7% agree) #Human

[Comment 67]: 🙂 Some local politicians abuse their power and get away with more than normal citizens. (424 votes, 63.2% agree) #Human

[Comment 222]: 🙂 Develop the Greenway so that the paved trail is continuous and reaches major parts of the city including the square, WKU, GM, and others. (487 votes, 68.8% agree) #Human

[Comment 201]: 🙂 Bowling Green needs more competitive internet rates. (498 votes, 93.2% agree) #Human

[Comment 606]: 🙂 Require businesses to pick up litter they and their customers generate. (180 votes, 85.6% agree) #Human

[Comment 347]: 🙂 Take advantage of the wonderful summers in KY and have more outdoor public swimming spaces (226 votes, 50.4% agree) #Human

[Comment 478]: 🙂 Complete the ky. 185 restructuring project. (55 votes, 49.1% agree) #Human

[Comment 859]: 🙂 Street lights should be required on residential streets in town. Too many dark corners. (76 votes, 68.4% agree) #Human

[Comment 88]: 🙂 The downtown district, especially Fountain Square, should have more retail businesses, not offices. (514 votes, 59.5% agree) #Human

[Comment 466]: 🙂 Bowling green needs to work in beautifying the city . More Boulvard like roads with planted flowers and reducing overhead utility clusters. (226 votes, 61.1% agree) #Human

[Comment 487]: 🙂 I see a lot of folks with expired tags on cars, both in state and out of state. Police need to enforce the laws and write more tickets. (98 votes, 38.8% agree) #Human

[Comment 341]: 🙂 People want it all, without paying for anything. A healthy community requires taxes people. (284 votes, 66.9% agree) #Human

[Comment 73]: 🙂 We need a health food store. (361 votes, 24.7% agree) #Human

[Comment 475]: 🙂 City/county parks should be focused on creating and maintain green places for residents to join.Avoid cement parking and adding buildings (193 votes, 61.7% agree) #Human

[Comment 76]: 🙂 Build and improve sidewalks in the area of Magnolia Street and 13th to Cabell Drive area for the heavy pedestrian traffic there. (414 votes, 66.2% agree) #Human

[Comment 460]: 🙂 Noise ordinances should be enacted and enforced in the county as they are in the city. (184 votes, 48.9% agree) #Human

[Comment 648]: 🙂 We need grocery store competition. Kroger doesn't even try because they don't have to. I'm sick of rotten produce. Let's get a Publix! (91 votes, 25.3% agree) #Human

[Comment 782]: 🙂 Build a homeless park w/tiny homes so they can live & not suffer. There should also be a main house to can receive/organize donations (55 votes, 12.7% agree) #Human

[Comment 523]: 🙂 Increase Internet/Cell/Phone service throughout the county.  First deal with areas that currently don't have any option. (231 votes, 85.3% agree) #Human

[Comment 499]: 🙂 Overdevelopment of rental property is detrimental to cities.  Renters do not pay property tax.  Schools suffer when the tax base is eroded. (134 votes, 26.9% agree) #Human

[Comment 109]: 🙂 The city and county school systems should merge. (296 votes, 12.2% agree) #Human

[Comment 386]: 🙂 Recruit and retain the best k-13 teachers graduating from WKU (264 votes, 87.9% agree) #Human

[Comment 639]: 🙂 Need engaging art installations throughout the city. People will be able to experience art, have fun, and share on social media. (106 votes, 43.4% agree) #Human

[Comment 654]: 🙂 Historic preservation of homes and buildings needs countywide attention to maintain our unique architectural heritage. (146 votes, 64.4% agree) #Human

[Comment 112]: 🙂 Local government, for profit and non profits should work together on housing and economic equality (535 votes, 70.5% agree) #Human

[Comment 258]: 🙂 Busy roads like the by pass either need turn lanes or block left hand turns except at stop lights. (458 votes, 63.8% agree) #Human

[Comment 755]: 🙂 If people are suggesting more advertising for tourists, there needs to be things for tourists to do. (75 votes, 78.7% agree) #Human

[Comment 505]: 🙂 Need to build wider access roads into the city! (86 votes, 39.5% agree) #Human

[Comment 624]: 🙂 Consistent, once a month testing of our emergency sirens, not just before potential bad weather. (72 votes, 61.1% agree) #Human

[Comment 392]: 🙂 More parking downtown.  There are too many reserved spaces which should be available in the evening. (314 votes, 70.1% agree) #Human

[Comment 115]: 🙂 The population has out grown the infrastructure in bg/ warren county. (454 votes, 67.4% agree) #Human

[Comment 776]: 🙂 BG needs to develop a marketing campaign to draw Warren Co and surrounding counties to downtown for eating, shopping, and entertainment. (82 votes, 53.7% agree) #Human

[Comment 636]: 🙂 Traffic congestion is a major problem, so studying traffic flow closely is the first step. (177 votes, 81.9% agree) #Human

[Comment 130]: 🙂 Only a traffic circle at Kereiakes Park on Fairview Ave. is required. (126 votes, 3.2% agree) #Human

[Comment 231]: 🙂 Bowling Green has many excellent non-public schools; more should be done to make these options available to diverse and low-income families. (288 votes, 11.1% agree) #Human

[Comment 642]: 🙂 More roundabouts for intersections (120 votes, 46.7% agree) #Human

[Comment 895]: 🙂 Smallhouse Rd (at Campbell Ln) heading into town needs to be widened to have room for three lanes (left turn, straight, right turn). (56 votes, 71.4% agree) #Human

[Comment 243]: 🙂 Funding and support should be given to the existing BG buses, to expand their service area and improve accessibility. (432 votes, 70.4% agree) #Human

[Comment 371]: 🙂 Bowling Green is an excellent place to raise a family, shop, worship, get an education, and recreate. I feel safe in most places here in BG. (330 votes, 86.1% agree) #Human

[Comment 773]: 🙂 WRECC should provide internet services. (71 votes, 69.0% agree) #Human

[Comment 127]: 🙂 City/County officials with connections to a project or proposal should abstain from voting on the project or proposal. (532 votes, 91.4% agree) #Human

[Comment 520]: 🙂 Tuckertown area needs fire protection. Currently rated Class 10 expensive insurance. Water lines are close by on Otter Gap & Tuckertown Rds. (31 votes, 35.5% agree) #Human

[Comment 237]: 🙂 Invest in the youth. (348 votes, 81.0% agree) #Human

[Comment 514]: 🙂 BG can’t compete with Nashville or Louisville for young professionals (104 votes, 28.8% agree) #Human

[Comment 362]: 🙂 BG Parks needs more funding  from City (148 votes, 13.5% agree) #Human

[Comment 502]: 🙂 Build more roads!!! Traffic is terrible here for a small city. (145 votes, 9.0% agree) #Human

[Comment 809]: 🙂 Better/more mass media choices available. (58 votes, 51.7% agree) #Human

[Comment 145]: 🙂 There should be greater checks on our County and City executives. They don't wield too much power but they do wield far too much influence. (266 votes, 63.2% agree) #Human

[Comment 684]: 🙂 Working folks need ordinances struck down that prevent them from assembling to bargain for better pay. benefits and working conditions. (89 votes, 37.1% agree) #Human

[Comment 32]: 🙂 We need a WHOLE FOOD store. (501 votes, 32.9% agree) #Human

[Comment 395]: 🙂 City residents do not receive adequate return on County Taxes paid. (102 votes, 7.8% agree) #Human

[Comment 5]: 🙂 Local law enforcement would benefit from additional training in community relations. (617 votes, 61.1% agree) #Human

[Comment 788]: 🙂 Pass a No animals on chains in the county. (60 votes, 3.3% agree) #Human

[Comment 142]: 🙂 There is not enough done for tourism in Warren Co. We offer much more than Corvettes and we should be promoting other activities. (575 votes, 64.2% agree) #Human

[Comment 404]: 🙂 Temporary housing for the homeless in the community. (261 votes, 64.0% agree) #Human

[Comment 279]: 🙂 New developments should help pay to widen the roads in the areas where they are increasing traffic. (447 votes, 83.0% agree) #Human

[Comment 535]: 🙂 We need to replace "Room at the Inn" services with temporary decent housing connected to aid agencies for the homeless (108 votes, 42.6% agree) #Human

[Comment 410]: 🙂 There are no public adult swim facilities in Bowling Green for year around physical use. The Lovers Lane Complex should be made available (189 votes, 71.4% agree) #Human

[Comment 803]: 🙂 Traffic flow on Cavemill Rd during peek hours (3pm - 5pm) (21 votes, 42.9% agree) #Human

[Comment 544]: 🙂 All new and existing sidewalks should have shade trees (123 votes, 8.9% agree) #Human

[Comment 815]: 🙂 Recycling services need to be improved and modernized. Current contractor uses outdated equipment, services are messy, company unresponsive. (107 votes, 64.5% agree) #Human

[Comment 657]: 🙂 More weekend activites are needed for YOUNG Seniors, who arent yet ready for Bingo nights. (103 votes, 63.1% agree) #Human

[Comment 139]: 🙂 Bowling Green City Commission needs to pass an ordinance banning the sale or use of large fireworks in the city limits. (354 votes, 8.5% agree) #Human

[Comment 11]: 🙂 Law enforcement should send drug possession arrestees for rehabilitation services rather than imprisonment. (719 votes, 43.5% agree) #Human

[Comment 538]: 🙂 Bowling Green  has been monopolized by Houchens.  Limit their growth they own too much of the city for it to be fair to other businesses (83 votes, 6.0% agree) #Human

[Comment 267]: 🙂 Higher minimum wage (593 votes, 12.3% agree) #Human

[Comment 163]: 🙂 Need more bike lanes and biker friendly resources (575 votes, 53.0% agree) #Human

[Comment 160]: 🙂 There is no reason not to spend 6 cents per $100 to eliminate homelessness for hundreds of school children. (309 votes, 37.9% agree) #Human

[Comment 529]: 🙂 Better public transit and runs 24 /7 so people without cars can get to work. Also more stops added in residential areas. (225 votes, 77.8% agree) #Human

[Comment 20]: 🙂 It is embarrassing that our city is the largest in the state not to have a fairness ordinance. (1022 votes, 11.0% agree) #Human

[Comment 273]: 🙂 I hope BG can one day link the short, scattered bicycle paths. (385 votes, 71.9% agree) #Human

[Comment 422]: 🙂 Public school could do alternate teaching styles(like Montessori) in a school within a school set up.  Not all kids learn the same way. (136 votes, 41.2% agree) #Human

[Comment 672]: 🙂 There is too much bullying in our schools that administrators are ignoring. (71 votes, 49.3% agree) #Human

[Comment 660]: 🙂 A traffic light is needed on Cave Mill in front of Natcher School.  Operational on school days, 7:30 to 9 AM & 3 to 4:30 PM. (60 votes, 3.3% agree) #Human

[Comment 276]: 🙂 Bike lanes that are wide enough to safely ride on without fear of being hit by passing cars. (420 votes, 70.0% agree) #Human

[Comment 157]: 🙂 We need more businesses like White Squirrel and Nats in BG. They add character to our town. Done with the Olive Garden and Belks. (420 votes, 47.6% agree) #Human

[Comment 526]: 🙂 Shade trees should be required plantings along all Greenways (107 votes, 51.4% agree) #Human

[Comment 416]: 🙂 Too many apartments in Plano. Nice little country setting I bought my home in forever ruined! (88 votes, 6.8% agree) #Human

[Comment 550]: 🙂 Greenwood Mall parking lot could become a cooperative solar collection for itself and home surrounding (95 votes, 38.9% agree) #Human

[Comment 282]: 🙂 Many drivers fail to stop as traffic lights turn red.  As many as 3 cars will travel through an intersection after light has changed to red. (252 votes, 69.8% agree) #Human

[Comment 17]: 🙂 If private schools were to get a share of Federal or State education dollars, it would hurt public schools. (632 votes, 35.1% agree) #Human

[Comment 419]: 🙂 some children with cultural differences or dietary needs cannot eat a typical lunch at school please provide special diet options. (162 votes, 12.3% agree) #Human

[Comment 413]: 🙂 BG police should improve officer training and should be held accountable for abuse of power. (154 votes, 64.9% agree) #Human

[Comment 663]: 🙂 WKU should open its facilities to the public once or twice a month to the community, and those in need should be welcome more often. (62 votes, 6.5% agree) #Human

[Comment 398]: 🙂 Bowling Green would be better if it didn't have the "Good Ole Boy" system among so many. (193 votes, 69.9% agree) #Human

[Comment 407]: 🙂 BG needs to install traffic cameras that help police intersections like Gallatin, TN.  This would reduce accidents, traffic flow and add $ (189 votes, 9.0% agree) #Human

[Comment 154]: 🙂 Every railroad crossing needs an elevating arm. Too many near accidents for something so inexpensive. (504 votes, 73.4% agree) #Human

[Comment 23]: 🙂 No child should go hungry in Bowling Green / Warren County, and it is up to our officials to make sure it doesn’t happen. (814 votes, 44.5% agree) #Human

[Comment 678]: 🙂 Doctors who prescribe pain and tranquilizing meds too easily fuel the drug dependency crisis. (106 votes, 54.7% agree) #Human

[Comment 148]: 🙂 The access roads off Scottsville Rd are terrible. They provide constant traffic jams and can be dangerous. We need to budget to fix them. (438 votes, 70.3% agree) #Human

[Comment 297]: 🙂 There should be something a work farm for local prisoners to learn to tend gardens, take care of livestock, and learn to be more useful. (300 votes, 52.7% agree) #Human

[Comment 565]: 🙂 We need well planned service roads, instead of so many driveways directly on streets like Campbell Lane and the By pass. (67 votes, 40.3% agree) #Human

[Comment 44]: 🙂 I would like to see a sift in city planning and zoning to make BG a more walkable city with better sidewalk planning (677 votes, 72.2% agree) #Human

[Comment 184]: 🙂 Install new sidewalks along major roads, like Three Springs, and city-wide bike routes, to decrease likelihood of accidents (546 votes, 61.9% agree) #Human

[Comment 574]: 🙂 Planning and zoning laws should change to encourage small businesses and farms. (158 votes, 63.3% agree) #Human

[Comment 589]: 🙂 Companies that claim they cannot fill job openings should raise wages and improve working conditions. (135 votes, 55.6% agree) #Human

[Comment 714]: 🙂 Drivers tests in BG should test over the rules of a turnabout. (137 votes, 72.3% agree) #Human

[Comment 827]: 🙂 Police should not focus on Public Intoxication charges for people walking home. We should encourage people to walk -- not drive drunk. (104 votes, 75.0% agree) #Human

[Comment 848]: 🙂 More parking for Fountain Square to become a pedestrian zone. (56 votes, 46.4% agree) #Human

[Comment 824]: 🙂 BGPD needs to work toward community policing, where cops patrol the neighborhoods they live in. Stop incentivizing cops to live in suburbs (58 votes, 51.7% agree) #Human

[Comment 437]: 🙂 Affordable supports for the elderly and aging population are needed in our community. (245 votes, 78.8% agree) #Human

[Comment 693]: 🙂 Bowling Green needs more accessible programs for mentorships of young adults and teenagers. (113 votes, 75.2% agree) #Human

[Comment 428]: 🙂 Bowling Green needs more indoor sports facilities, particularly a large complex with multiple indoor soccer fields, for youth. (156 votes, 2.6% agree) #Human

[Comment 446]: 🙂 A fresh food grocery store to be built in Delafield and Morgantown rd areas within walking distance of residents. (140 votes, 58.6% agree) #Human

[Comment 190]: 🙂 Institute a business development grant that encourages WKU grads to stay in Bowling Green by starting local businesses. (476 votes, 63.4% agree) #Human

[Comment 458]: 🙂 Oversight committees for projects such as TIF. Elected officials should not be able to sit on unelected boards. (144 votes, 79.2% agree) #Human

[Comment 690]: 🙂 Holly drive needs sidewalks (40 votes, 25.0% agree) #Human

[Comment 821]: 🙂 Low-income areas should get beautification money -- not the wealthy who can afford it themselves. (92 votes, 73.9% agree) #Human

[Comment 50]: 🙂 A person should not miss rent (say due to a car breakdown) and face eviction in less than 2 weeks (before next payday). Make it 30 days. (436 votes, 38.5% agree) #Human

[Comment 196]: 🙂 Bowling Green needs to attract business like Costco and Trader Joe's to keep residents from driving to Nashville on weekends (564 votes, 69.1% agree) #Human

[Comment 41]: 🙂 Fewer students per teacher should be a rule. No more than 20; ideal class size of 15. Plenty of teachers are looking for work. (465 votes, 36.8% agree) #Human

[Comment 65]: 🙂 Prevent the development of new housing areas, especially in farm land areas. (356 votes, 5.6% agree) #Human

[Comment 172]: 🙂 The road closure between Shawnee and Patton Way makes access to Nashville Rd take 3 times as long and is a major annoyance. (70 votes, 22.9% agree) #Human

[Comment 699]: 🙂 Parking on the WKU campus should be more accessible to students and visitors. (123 votes, 78.9% agree) #Human

[Comment 702]: 🙂 Arts education should be available in every school in Warren County and other surrounding areas. (155 votes, 80.6% agree) #Human

[Comment 56]: 🙂 We need more things to do for ages 1-6. A children’s museum or a small science museum would be wonderful. (598 votes, 62.5% agree) #Human

[Comment 452]: 🙂 Consider asking low-income or disabled residents what they really need. (191 votes, 82.2% agree) #Human

[Comment 696]: 🙂 There should be more healthy food options near/on campus of WKU. (97 votes, 52.6% agree) #Human

[Comment 577]: 🙂 the Bark Park needs more trees and water access for the smaller dogs area (55 votes, 52.7% agree) #Human

[Comment 35]: 🙂 I would like to see more community art and music programs. For instance, I would love to sign up for a community choir. (532 votes, 41.4% agree) #Human

[Comment 187]: 🙂 Better incorporation of university resources in community, including Kentucky Museum. (412 votes, 71.8% agree) #Human

[Comment 47]: 🙂 Continue downtown development to the river.  Tourists always seek out parks & restaurants near water. (747 votes, 78.3% agree) #Human

[Comment 178]: 🙂 It would be great if people in neighborhoods could get to know each other better and neighborhoods were like small communities of neighbors. (384 votes, 79.2% agree) #Human

[Comment 580]: 🙂 Wal-Mart and Kroger grocery pick-up should have to accept EBT for the elderly and disabled (92 votes, 34.8% agree) #Human

[Comment 38]: 🙂 All schools in the county and city should work together to provide the same minimum level of education to students. (487 votes, 76.6% agree) #Human

[Comment 300]: 🙂 Parking lots should be monitored to prevent illegal parking in handicap spaces. (182 votes, 50.5% agree) #Human

[Comment 318]: 🙂 Traffic congestion on Fairview Ave and Cemetery Rd needs to be improved during evening rush hours. (325 votes, 80.9% agree) #Human

[Comment 455]: 🙂 Regulate the size & brightness of business signs. (144 votes, 48.6% agree) #Human

[Comment 181]: 🙂 The university and city should develop stronger mutually beneficial partnerships. (415 votes, 77.8% agree) #Human

[Comment 440]: 🙂 We need more mixed-use developments so people live closely to all basic needs. Mix business and residential zones and increase walkability. (222 votes, 72.1% agree) #Human

[Comment 839]: 🙂 Need term limits for all local elected officiałs (116 votes, 69.0% agree) #Human

[Comment 741]: 🙂 The bypass area must be developed and utilized for new businesses and attractive structures. It is a neglected area that is losing business. (169 votes, 78.7% agree) #Human

[Comment 750]: 🙂 We need to offer free full panel STD testing to students and low income families (103 votes, 24.3% agree) #Human

[Comment 872]: 🙂 We need faster internet service. (94 votes, 83.0% agree) #Human

[Comment 476]: 🙂 We have an estimated 125,000 people but we still don't have a hospital that is trauma center and have to ship everyone to TN or Louisville. (154 votes, 74.0% agree) #Human

[Comment 470]: 🙂 It is easy to find a good job in Warren County. (64 votes, 3.1% agree) #Human

[Comment 753]: 🙂 We need more regulation of opioid drugs, both prescribed and illicit. (82 votes, 41.5% agree) #Human

[Comment 214]: 🙂 More bike trails and walking trails would be a great addition. (613 votes, 71.6% agree) #Human

[Comment 336]: 🙂 With drug use being so prevalent in Warren county, better treatment options need to be offered. (358 votes, 73.7% agree) #Human

[Comment 95]: 🙂 Downtown at the square and adjacent areas should be a pedestrian mall (360 votes, 23.9% agree) #Human

[Comment 86]: 🙂 Bowling Green needs a fairness ordinance to protect the rights of its citizens. (990 votes, 19.4% agree) #Human

[Comment 744]: 🙂 We need more activities/places to go for teenagers; basically all there is are places to eat, the mall, and the bowling alley (78 votes, 59.0% agree) #Human

[Comment 488]: 🙂 When the city tears up lawns with projects they should be required to do what it takes so that lawns are same quality they were before. (213 votes, 90.6% agree) #Human

[Comment 202]: 🙂 Residential Parking Permits for areas near WKU. We can rescue our yards from parking. We pay property taxes for the students' convenience, (230 votes, 55.7% agree) #Human

[Comment 610]: 🙂 There should be no punishment for small amounts of marijuana possession, in my opinion (132 votes, 12.1% agree) #Human

[Comment 77]: 🙂 Bring a discount grocer to the downtown area. (321 votes, 45.8% agree) #Human

[Comment 860]: 🙂 New buildings should not be built right up to the sidewalk to inhibit seeing around corners when driving. (105 votes, 71.4% agree) #Human

[Comment 485]: 🙂 Police do a shameful job of enforcing traffic laws. more tickets = more compliance and more $$$$$. (105 votes, 2.9% agree) #Human

[Comment 229]: 🙂 Internet services should be a public municipality. (387 votes, 55.0% agree) #Human

[Comment 71]: 🙂 Greenwood Mall should be remodeled to attract more buyers. (448 votes, 17.9% agree) #Human

[Comment 205]: 🙂 Refugees and immigrants are welcome next door. (388 votes, 37.1% agree) #Human

[Comment 738]: 🙂 Educate high schoolers on their options after they graduate (134 votes, 85.1% agree) #Human

[Comment 473]: 🙂 Higher paying jobs- the ability to move up with reasonable raises. (147 votes, 67.3% agree) #Human

[Comment 729]: 🙂 More green spaces. All of our parks have just become sports complexes! (145 votes, 53.1% agree) #Human

[Comment 875]: 🙂 Older neighborhoods change from owner occupied to rental.  Deterioration occurs.  Stronger landlord/tenant laws needed! (84 votes, 57.1% agree) #Human

[Comment 220]: 🙂 Stronger tenant and renters rights. Landlord accountability for keeping properties in good shape. So many overpriced dilapidated rentals. (480 votes, 62.9% agree) #Human

[Comment 735]: 🙂 Schools should offer more comprehensive sexual education to help lower STD and teen pregnancy rates. (74 votes, 48.6% agree) #Human

[Comment 348]: 🙂 Extra police, or parking enforcement unit should be on duty during events to maintain legal parking (222 votes, 38.7% agree) #Human

[Comment 595]: 🙂 The LEFT turning light at Nashville onto Dishman & the same for Scottsville onto Lovers Ln.. need to be longer to allow more to go through. (60 votes, 33.3% agree) #Human

[Comment 345]: 🙂 Make all future sidewalks wide enough for multi-use (walking, biking, skateboarding etc) (289 votes, 44.6% agree) #Human

[Comment 339]: 🙂 city needs to work on water run off when we have a lot of rain.. to many places flood on roadways makes driving very dangerous.. (326 votes, 80.4% agree) #Human

[Comment 83]: 🙂 Impact on traffic flow should be considered with any proposed new developments. (676 votes, 94.1% agree) #Human

[Comment 863]: 🙂 SNAP grants for neighborhood improvements should be available in Warren County, not just in the city. (45 votes, 55.6% agree) #Human

[Comment 68]: 🙂 County volunteer fire departments need to be upgraded to fit the current population of Warren County. (571 votes, 81.8% agree) #Human

[Comment 223]: 🙂 Rent is too expensive inside the city limits. (204 votes, 41.2% agree) #Human

[Comment 226]: 🙂 The WKU Nursing Program should be expanded so we have more nurses (140 votes, 58.6% agree) #Human

[Comment 467]: 🙂 Work,with State to complete a wide loop from Scottsville Road across N I65 to N Natcher Pk way. (63 votes, 14.3% agree) #Human

[Comment 857]: 🙂 BG's Kummer Little gym is open limited hours for indoor track. On other hand, County has new gyms that are open lots of hrs. Inequitable. (30 votes, 60.0% agree) #Human

[Comment 357]: 🙂 An unmanned trolley system, possibly solar, might be better long term than buses for some areas. (169 votes, 18.3% agree) #Human

[Comment 866]: 🙂 A round about would be the best solution to improve traffic flow on Fairview Ave at Keriakes Park. (45 votes, 42.2% agree) #Human

[Comment 208]: 🙂 BG has a bad problem with littering.  My husband and I fill up to 5 recycle bins every week between downtown and campus, cleaning it all up. (327 votes, 79.2% agree) #Human

[Comment 869]: 🙂 Insist that schools at all levels have daily physical activity  for all students to help combat the childhood obesity crisis. (102 votes, 86.3% agree) #Human

[Comment 381]: 🙂 Smoking should not be allowed within 25 feet of a business. (235 votes, 37.0% agree) #Human

[Comment 494]: 🙂 We need a fiber option for internet. (221 votes, 93.7% agree) #Human

[Comment 104]: 🙂 Downtown redevelopment should retain accessible parking for existing businesses. (528 votes, 87.9% agree) #Human

[Comment 774]: 🙂 Bowling Green Government needs more perspective from Millennials. We need ideas for the future not outdated Boomer development ideas. (79 votes, 24.1% agree) #Human

[Comment 640]: 🙂 More healthy restaurants (81 votes, 50.6% agree) #Human

[Comment 101]: 🙂 We need jobs to attract our own college educated students so they don’t leave to get better paying jobs (588 votes, 85.4% agree) #Human

[Comment 890]: 🙂 More affordable home ownership options. (70 votes, 80.0% agree) #Human

[Comment 107]: 🙂 We are loosing the city’s long established neighborhoods to multi family developments that do not fit in with the single family residences (496 votes, 49.2% agree) #Human

[Comment 649]: 🙂 Planning & Zoning Board needs an overhaul. Fewer builders appointed, fairness to opposition, stop rubber stamping by P&Z and City Comm. (117 votes, 62.4% agree) #Human

[Comment 232]: 🙂 City had 109 homeless students in 2017, County Had 112.  Housing those families temporarily is worth raising tax rate from 1.85% to 1.91%. (382 votes, 24.1% agree) #Human

[Comment 250]: 🙂 When you prioritize the safety of people over automobile speed, all kinds of positive things happen. #slowthecars (214 votes, 48.6% agree) #Human

[Comment 366]: 🙂 Too many trees have been torn down to make way for new developments. We should re-plant more trees around Bowling Green. (412 votes, 84.0% agree) #Human

[Comment 771]: 🙂 There needs to be a stoplight at the intersection of Shive Lane and Ken Bale Blvd to improve traffic flow. The backups there are awful. (53 votes, 5.7% agree) #Human

[Comment 262]: 🙂 More technical education programs for non-graduates of high school (418 votes, 74.2% agree) #Human

[Comment 631]: 🙂 City/county leaders should be cautious of "build it and they will come" mentality for growth and development. (98 votes, 69.4% agree) #Human

[Comment 244]: 🙂 Develop a manufacturing Work training program for jail inmates that qualify (364 votes, 79.7% agree) #Human

[Comment 122]: 🙂 Driver’s education needs to be more thorough and mandatory before 16 year olds are even allowed to drive home from the courthouse. (390 votes, 48.7% agree) #Human

[Comment 506]: 🙂 City officials need to work with WKU admin. to address the noise & public nuisance problems with the frat houses and homeowners downtown. (106 votes, 39.6% agree) #Human

[Comment 759]: 🙂 Provide a food assistance program for college students as students lots of times do not qualify for food stamps due to living on campus (74 votes, 8.1% agree) #Human

[Comment 643]: 🙂 Commitment to healthy lifestyles (78 votes, 64.1% agree) #Human

[Comment 524]: 🙂 Urban sprawl in Bowling Green is a major problem that has made this city less walking/bike friendly. (119 votes, 49.6% agree) #Human

[Comment 247]: 🙂 competition for cable companies and an end to heir price gouging. (460 votes, 93.9% agree) #Human

[Comment 500]: 🙂 Need comprehesive probation rehab programs - include addiction treatment, life & job skills & ideas securing housing, training/job etc (159 votes, 63.5% agree) #Human

[Comment 241]: 🙂 We need to focus more on the existing housing in BG, including renovations and improvements. (404 votes, 79.2% agree) #Human

[Comment 128]: 🙂 New developments should not be approved unless roads and infrastructure capable of handling them are in place. (567 votes, 86.9% agree) #Human

[Comment 777]: 🙂 A fine if your cat or dog is picked up as a stray and is not spayed or neutered in the county. (82 votes, 22.0% agree) #Human

[Comment 110]: 🙂 Trains blocking major roads, stopping, and shutting down all traffic. This problem needs to be solved. (472 votes, 45.3% agree) #Human

[Comment 625]: 🙂 Future charter schools would be a financial burden to public school funding, unless they are held to accountability by local school boards. (72 votes, 33.3% agree) #Human

[Comment 131]: 🙂 With a circle, widen Fairview with a left-turn lane west of cemetery. (142 votes, 47.9% agree) #Human

[Comment 256]: 🙂 BG needs more Summer weekend festivals/activities (408 votes, 65.7% agree) #Human

[Comment 113]: 🙂 City/County schools should have a minimum one year classroom for immigrant and refugee students so they can learn english, culture and laws (488 votes, 38.9% agree) #Human

[Comment 634]: 🙂 City/county gov't. (in all of there forms) must reflect the racial/ethnic city/county population demographic to assist in recruitment/growth (104 votes, 15.4% agree) #Human

<Tenant Support in Downtown District>
  + [Comment 875]
  + [Comment 641]
  + [Comment 436]
  + [Comment 405]
  + [Comment 223]
  + [Comment 213]
  + [Comment 140]
  + [Comment 144]
  + [Comment 112]
  + [Comment 30]

<Improved Traffic Flow and Safety>
  + [Comment 895]
  + [Comment 888]
  + [Comment 866]
  + [Comment 860]
  + [Comment 805]
  + [Comment 804]
  + [Comment 803]
  + [Comment 798]
  + [Comment 771]
  + [Comment 660]
  + [Comment 642]
  + [Comment 636]
  + [Comment 596]
  + [Comment 595]
  + [Comment 565]
  + [Comment 531]
  + [Comment 502]
  + [Comment 486]
  + [Comment 483]
  + [Comment 435]
  + [Comment 415]
  + [Comment 411]
  + [Comment 407]
  + [Comment 394]
  + [Comment 718]
  + [Comment 714]
  + [Comment 352]
  + [Comment 340]
  + [Comment 318]
  + [Comment 293]
  + [Comment 283]
  + [Comment 282]
  + [Comment 279]
  + [Comment 258]
  + [Comment 250]
  + [Comment 227]
  + [Comment 209]
  + [Comment 172]
  + [Comment 478]
  + [Comment 468]
  + [Comment 467]
  + [Comment 168]
  + [Comment 156]
  + [Comment 153]
  + [Comment 149]
  + [Comment 148]
  + [Comment 131]
  + [Comment 130]
  + [Comment 128]
  + [Comment 122]
  + [Comment 83]
  + [Comment 82]
  + [Comment 69]
  + [Comment 66]

<Regulate over-prescribing doctors>
  + [Comment 688]
  + [Comment 683]
  + [Comment 679]
  + [Comment 678]
  + [Comment 500]
  + [Comment 753]
  + [Comment 742]
  + [Comment 19]
  + [Comment 11]
  + [Comment 10]

<Enhance addiction treatment and rehabilitation>
  + [Comment 678]
  + [Comment 500]
  + [Comment 752]
  + [Comment 742]
  + [Comment 336]
  + [Comment 245]
  + [Comment 19]
  + [Comment 11]
  + [Comment 10]

<Address drug addiction as a health issue>
  + [Comment 894]
  + [Comment 500]
  + [Comment 753]
  + [Comment 752]
  + [Comment 742]
  + [Comment 709]
  + [Comment 336]
  + [Comment 245]
  + [Comment 19]
  + [Comment 11]
  + [Comment 10]

<Equitable School Redistricting>
  + [Comment 584]
  + [Comment 498]
  + [Comment 430]
  + [Comment 422]
  + [Comment 743]
  + [Comment 263]
  + [Comment 113]
  + [Comment 109]
  + [Comment 41]
  + [Comment 39]
  + [Comment 38]

<Consistent Education Standards>
  + [Comment 869]
  + [Comment 801]
  + [Comment 545]
  + [Comment 498]
  + [Comment 422]
  + [Comment 736]
  + [Comment 236]
  + [Comment 113]
  + [Comment 109]
  + [Comment 41]
  + [Comment 39]
  + [Comment 38]
  + [Comment 21]

<Healthier Community>
  + [Comment 891]
  + [Comment 542]
  + [Comment 410]
  + [Comment 744]
  + [Comment 287]
  + [Comment 72]

<Expanding Access to Resources>
  + [Comment 885]
  + [Comment 863]
  + [Comment 796]
  + [Comment 724]
  + [Comment 115]

<Clean Yards>
  + [Comment 689]
  + [Comment 673]
  + [Comment 606]
  + [Comment 578]
  + [Comment 557]
  + [Comment 414]
  + [Comment 308]
  + [Comment 208]

<Enhanced transportation accessibility>
  + [Comment 870]
  + [Comment 868]
  + [Comment 588]
  + [Comment 579]
  + [Comment 564]
  + [Comment 529]
  + [Comment 357]
  + [Comment 356]
  + [Comment 268]
  + [Comment 243]
  + [Comment 215]
  + [Comment 57]

<Diversity in City/County Government>
  + [Comment 668]
  + [Comment 634]
  + [Comment 581]
  + [Comment 535]
  + [Comment 452]
  + [Comment 413]
  + [Comment 299]
  + [Comment 266]
  + [Comment 260]
  + [Comment 220]
  + [Comment 205]
  + [Comment 144]
  + [Comment 112]
  + [Comment 61]
  + [Comment 34]
  + [Comment 30]
  + [Comment 5]
  + [Comment 4]
  + [Comment 3]

<Improve medical care access>
  + [Comment 846]
  + [Comment 679]
  + [Comment 500]
  + [Comment 444]
  + [Comment 752]
  + [Comment 336]
  + [Comment 476]
  + [Comment 10]

<Essential Education Funding>
  + [Comment 869]
  + [Comment 801]
  + [Comment 799]
  + [Comment 545]
  + [Comment 498]
  + [Comment 433]
  + [Comment 430]
  + [Comment 422]
  + [Comment 419]
  + [Comment 751]
  + [Comment 738]
  + [Comment 736]
  + [Comment 710]
  + [Comment 702]
  + [Comment 701]
  + [Comment 386]
  + [Comment 237]
  + [Comment 236]
  + [Comment 113]
  + [Comment 41]
  + [Comment 39]
  + [Comment 38]
  + [Comment 35]
  + [Comment 21]

<Charter School Accountability>
  + [Comment 625]
  + [Comment 430]
  + [Comment 421]
  + [Comment 751]
  + [Comment 736]
  + [Comment 41]
  + [Comment 39]
  + [Comment 38]
  + [Comment 17]

<Youth Investment>
  + [Comment 869]
  + [Comment 834]
  + [Comment 801]
  + [Comment 799]
  + [Comment 672]
  + [Comment 584]
  + [Comment 554]
  + [Comment 551]
  + [Comment 545]
  + [Comment 498]
  + [Comment 433]
  + [Comment 430]
  + [Comment 422]
  + [Comment 421]
  + [Comment 419]
  + [Comment 751]
  + [Comment 743]
  + [Comment 738]
  + [Comment 736]
  + [Comment 735]
  + [Comment 710]
  + [Comment 703]
  + [Comment 702]
  + [Comment 701]
  + [Comment 386]
  + [Comment 263]
  + [Comment 262]
  + [Comment 237]
  + [Comment 236]
  + [Comment 113]
  + [Comment 109]
  + [Comment 101]
  + [Comment 41]
  + [Comment 39]
  + [Comment 38]
  + [Comment 21]
  + [Comment 16]

<Active Community>
  + [Comment 891]
  + [Comment 857]
  + [Comment 542]
  + [Comment 428]
  + [Comment 410]
  + [Comment 400]
  + [Comment 744]
  + [Comment 287]
  + [Comment 221]
  + [Comment 72]
  + [Comment 37]

<Improving Pedestrian Safety>
  + [Comment 807]
  + [Comment 690]
  + [Comment 544]
  + [Comment 453]
  + [Comment 345]
  + [Comment 276]
  + [Comment 251]
  + [Comment 222]
  + [Comment 214]
  + [Comment 203]
  + [Comment 184]
  + [Comment 146]
  + [Comment 95]
  + [Comment 76]
  + [Comment 47]

<Improving Pedestrian Safety on Major Roads>
  + [Comment 690]
  + [Comment 544]
  + [Comment 453]
  + [Comment 345]
  + [Comment 326]
  + [Comment 251]
  + [Comment 222]
  + [Comment 214]
  + [Comment 203]
  + [Comment 184]
  + [Comment 146]
  + [Comment 95]
  + [Comment 76]

<Modernizing Emergency Services>
  + [Comment 520]
  + [Comment 519]
  + [Comment 115]
  + [Comment 68]

<Economic Benefits>
  + [Comment 610]
  + [Comment 129]
  + [Comment 85]
  + [Comment 13]
  + [Comment 12]

<Improved Traffic Enforcement>
  + [Comment 895]
  + [Comment 888]
  + [Comment 860]
  + [Comment 804]
  + [Comment 803]
  + [Comment 798]
  + [Comment 660]
  + [Comment 642]
  + [Comment 624]
  + [Comment 596]
  + [Comment 595]
  + [Comment 585]
  + [Comment 531]
  + [Comment 486]
  + [Comment 485]
  + [Comment 484]
  + [Comment 483]
  + [Comment 435]
  + [Comment 407]
  + [Comment 718]
  + [Comment 714]
  + [Comment 352]
  + [Comment 318]
  + [Comment 311]
  + [Comment 293]
  + [Comment 283]
  + [Comment 282]
  + [Comment 279]
  + [Comment 258]
  + [Comment 250]
  + [Comment 227]
  + [Comment 156]
  + [Comment 153]
  + [Comment 149]
  + [Comment 148]
  + [Comment 128]
  + [Comment 122]
  + [Comment 83]
  + [Comment 82]
  + [Comment 69]
  + [Comment 66]

<Modernizing Urban Development>
  + [Comment 861]
  + [Comment 775]
  + [Comment 774]
  + [Comment 695]
  + [Comment 524]
  + [Comment 366]
  + [Comment 196]
  + [Comment 466]
  + [Comment 171]
  + [Comment 87]

<Parking Regulation Enforcement>
  + [Comment 848]
  + [Comment 847]
  + [Comment 561]
  + [Comment 558]
  + [Comment 713]
  + [Comment 699]
  + [Comment 348]
  + [Comment 300]
  + [Comment 204]
  + [Comment 202]
  + [Comment 104]

<Culturally Enriched City>
  + [Comment 639]
  + [Comment 635]
  + [Comment 608]
  + [Comment 744]
  + [Comment 256]
  + [Comment 221]
  + [Comment 187]
  + [Comment 181]
  + [Comment 158]
  + [Comment 138]
  + [Comment 70]
  + [Comment 37]

<Boosting Tourism and Economy>
  + [Comment 776]
  + [Comment 320]
  + [Comment 142]
  + [Comment 99]

<Bridging the Digital Divide>
  + [Comment 872]
  + [Comment 773]
  + [Comment 523]
  + [Comment 494]
  + [Comment 423]
  + [Comment 715]
  + [Comment 295]
  + [Comment 229]
  + [Comment 228]
  + [Comment 155]
  + [Comment 64]

<Expanded Nashville transportation>
  + [Comment 870]
  + [Comment 868]
  + [Comment 588]
  + [Comment 579]
  + [Comment 564]
  + [Comment 529]
  + [Comment 357]
  + [Comment 356]
  + [Comment 268]
  + [Comment 243]
  + [Comment 215]
  + [Comment 57]

<Improved Traffic Flow>
  + [Comment 895]
  + [Comment 882]
  + [Comment 866]
  + [Comment 805]
  + [Comment 804]
  + [Comment 803]
  + [Comment 798]
  + [Comment 642]
  + [Comment 636]
  + [Comment 596]
  + [Comment 565]
  + [Comment 502]
  + [Comment 483]
  + [Comment 435]
  + [Comment 415]
  + [Comment 411]
  + [Comment 394]
  + [Comment 718]
  + [Comment 714]
  + [Comment 318]
  + [Comment 293]
  + [Comment 279]
  + [Comment 258]
  + [Comment 250]
  + [Comment 227]
  + [Comment 209]
  + [Comment 478]
  + [Comment 467]
  + [Comment 156]
  + [Comment 153]
  + [Comment 149]
  + [Comment 148]
  + [Comment 130]
  + [Comment 128]
  + [Comment 83]
  + [Comment 82]
  + [Comment 69]
  + [Comment 66]

<Community Policing Improvement>
  + [Comment 824]
  + [Comment 697]

<Green Path and Parklike Spaces>
  + [Comment 847]
  + [Comment 558]
  + [Comment 550]
  + [Comment 533]
  + [Comment 202]
  + [Comment 191]

<Comprehensive Sex Education>
  + [Comment 545]
  + [Comment 422]
  + [Comment 751]
  + [Comment 750]
  + [Comment 738]
  + [Comment 735]
  + [Comment 237]
  + [Comment 236]
  + [Comment 39]
  + [Comment 38]

<Improving Downtown Pedestrian Experience>
  + [Comment 690]
  + [Comment 544]
  + [Comment 453]
  + [Comment 345]
  + [Comment 276]
  + [Comment 251]
  + [Comment 222]
  + [Comment 214]
  + [Comment 184]
  + [Comment 146]
  + [Comment 95]
  + [Comment 76]
  + [Comment 47]

<Bridging the Digital Divide through Free WiFi>
  + [Comment 773]
  + [Comment 523]
  + [Comment 423]
  + [Comment 715]
  + [Comment 295]
  + [Comment 229]
  + [Comment 228]
  + [Comment 155]
  + [Comment 64]

<Improved Nashville Airport transport>
  + [Comment 870]
  + [Comment 868]
  + [Comment 588]
  + [Comment 579]
  + [Comment 564]
  + [Comment 529]
  + [Comment 356]
  + [Comment 268]
  + [Comment 243]
  + [Comment 215]
  + [Comment 57]

<Strong Landlord-Tenant Laws>
  + [Comment 875]
  + [Comment 822]
  + [Comment 788]
  + [Comment 777]
  + [Comment 582]
  + [Comment 581]
  + [Comment 572]
  + [Comment 535]
  + [Comment 499]
  + [Comment 452]
  + [Comment 436]
  + [Comment 409]
  + [Comment 405]
  + [Comment 260]
  + [Comment 232]
  + [Comment 223]
  + [Comment 220]
  + [Comment 213]
  + [Comment 212]
  + [Comment 207]
  + [Comment 206]
  + [Comment 176]
  + [Comment 459]
  + [Comment 144]
  + [Comment 112]
  + [Comment 54]
  + [Comment 50]
  + [Comment 34]
  + [Comment 30]

<Rental Assistance and Affordability>
  + [Comment 875]
  + [Comment 822]
  + [Comment 782]
  + [Comment 581]
  + [Comment 535]
  + [Comment 452]
  + [Comment 437]
  + [Comment 436]
  + [Comment 409]
  + [Comment 341]
  + [Comment 260]
  + [Comment 232]
  + [Comment 223]
  + [Comment 220]
  + [Comment 213]
  + [Comment 176]
  + [Comment 459]
  + [Comment 160]
  + [Comment 147]
  + [Comment 144]
  + [Comment 112]
  + [Comment 50]
  + [Comment 34]

<Enforcing Noise Ordinances>
  + [Comment 861]
  + [Comment 695]
  + [Comment 506]
  + [Comment 379]
  + [Comment 371]
  + [Comment 174]
  + [Comment 460]
  + [Comment 170]
  + [Comment 159]
  + [Comment 139]

<Equitable Resource Distribution>
  + [Comment 832]
  + [Comment 821]
  + [Comment 820]
  + [Comment 819]
  + [Comment 659]
  + [Comment 653]
  + [Comment 649]
  + [Comment 631]
  + [Comment 577]
  + [Comment 574]
  + [Comment 534]
  + [Comment 513]
  + [Comment 441]
  + [Comment 440]
  + [Comment 412]
  + [Comment 755]
  + [Comment 731]
  + [Comment 729]
  + [Comment 700]
  + [Comment 362]
  + [Comment 301]
  + [Comment 275]
  + [Comment 241]
  + [Comment 192]
  + [Comment 189]
  + [Comment 185]
  + [Comment 183]
  + [Comment 182]
  + [Comment 179]
  + [Comment 178]
  + [Comment 475]
  + [Comment 111]
  + [Comment 88]
  + [Comment 60]
  + [Comment 58]
  + [Comment 44]
  + [Comment 18]

<Enhanced Youth Development>
  + [Comment 891]
  + [Comment 857]
  + [Comment 693]
  + [Comment 639]
  + [Comment 608]
  + [Comment 542]
  + [Comment 428]
  + [Comment 400]
  + [Comment 744]
  + [Comment 231]
  + [Comment 221]
  + [Comment 181]
  + [Comment 180]
  + [Comment 158]
  + [Comment 37]
  + [Comment 15]

<Enhancing Safety and Quality of Life>
  + [Comment 520]
  + [Comment 519]
  + [Comment 730]
  + [Comment 115]
  + [Comment 99]
  + [Comment 68]

<Modern Recycling>
  + [Comment 816]
  + [Comment 815]
  + [Comment 689]
  + [Comment 673]
  + [Comment 578]
  + [Comment 557]
  + [Comment 536]
  + [Comment 414]
  + [Comment 308]
  + [Comment 302]
  + [Comment 208]

<Police Presence in County Areas>
  + [Comment 668]
  + [Comment 563]
  + [Comment 487]
  + [Comment 341]
  + [Comment 5]

<Improved Traffic Control Measures>
  + [Comment 895]
  + [Comment 888]
  + [Comment 882]
  + [Comment 866]
  + [Comment 860]
  + [Comment 859]
  + [Comment 805]
  + [Comment 804]
  + [Comment 803]
  + [Comment 798]
  + [Comment 771]
  + [Comment 660]
  + [Comment 642]
  + [Comment 636]
  + [Comment 596]
  + [Comment 595]
  + [Comment 585]
  + [Comment 565]
  + [Comment 531]
  + [Comment 505]
  + [Comment 502]
  + [Comment 486]
  + [Comment 484]
  + [Comment 483]
  + [Comment 435]
  + [Comment 415]
  + [Comment 411]
  + [Comment 407]
  + [Comment 394]
  + [Comment 718]
  + [Comment 714]
  + [Comment 352]
  + [Comment 340]
  + [Comment 318]
  + [Comment 311]
  + [Comment 293]
  + [Comment 283]
  + [Comment 282]
  + [Comment 279]
  + [Comment 258]
  + [Comment 250]
  + [Comment 227]
  + [Comment 209]
  + [Comment 172]
  + [Comment 478]
  + [Comment 467]
  + [Comment 168]
  + [Comment 156]
  + [Comment 154]
  + [Comment 153]
  + [Comment 149]
  + [Comment 148]
  + [Comment 131]
  + [Comment 130]
  + [Comment 128]
  + [Comment 122]
  + [Comment 83]
  + [Comment 82]
  + [Comment 69]
  + [Comment 66]

<Improved Road Aesthetics>
  + [Comment 895]
  + [Comment 888]
  + [Comment 882]
  + [Comment 860]
  + [Comment 859]
  + [Comment 805]
  + [Comment 804]
  + [Comment 642]
  + [Comment 596]
  + [Comment 565]
  + [Comment 483]
  + [Comment 435]
  + [Comment 411]
  + [Comment 741]
  + [Comment 364]
  + [Comment 340]
  + [Comment 316]
  + [Comment 293]
  + [Comment 286]
  + [Comment 279]
  + [Comment 250]
  + [Comment 478]
  + [Comment 467]
  + [Comment 168]
  + [Comment 153]
  + [Comment 149]
  + [Comment 148]
  + [Comment 128]

<Parking Program Improvement>
  + [Comment 848]
  + [Comment 847]
  + [Comment 558]
  + [Comment 550]
  + [Comment 546]
  + [Comment 533]
  + [Comment 450]
  + [Comment 713]
  + [Comment 699]
  + [Comment 348]
  + [Comment 322]
  + [Comment 300]
  + [Comment 204]
  + [Comment 202]
  + [Comment 191]
  + [Comment 104]

<Food Assistance Program>
  + [Comment 696]
  + [Comment 663]
  + [Comment 759]

<Empowering Mentorship>
  + [Comment 891]
  + [Comment 693]
  + [Comment 635]
  + [Comment 608]
  + [Comment 428]
  + [Comment 748]
  + [Comment 744]
  + [Comment 287]
  + [Comment 231]
  + [Comment 226]
  + [Comment 221]
  + [Comment 187]
  + [Comment 181]
  + [Comment 180]
  + [Comment 43]
  + [Comment 37]
  + [Comment 15]

<Engaged University-Community Partnership>
  + [Comment 693]
  + [Comment 639]
  + [Comment 608]
  + [Comment 748]
  + [Comment 226]
  + [Comment 221]
  + [Comment 190]
  + [Comment 187]
  + [Comment 181]
  + [Comment 56]
  + [Comment 43]
  + [Comment 37]

<Improving Cycling Experience>
  + [Comment 784]
  + [Comment 526]
  + [Comment 453]
  + [Comment 345]
  + [Comment 326]
  + [Comment 276]
  + [Comment 273]
  + [Comment 251]
  + [Comment 222]
  + [Comment 218]
  + [Comment 214]
  + [Comment 184]
  + [Comment 163]
  + [Comment 95]

<Responsible Businesses>
  + [Comment 816]
  + [Comment 689]
  + [Comment 673]
  + [Comment 606]
  + [Comment 578]
  + [Comment 557]
  + [Comment 414]
  + [Comment 308]
  + [Comment 302]
  + [Comment 208]

<Improved daily commuting>
  + [Comment 870]
  + [Comment 868]
  + [Comment 588]
  + [Comment 579]
  + [Comment 564]
  + [Comment 529]
  + [Comment 356]
  + [Comment 268]
  + [Comment 243]
  + [Comment 215]
  + [Comment 57]

<Real bus system>
  + [Comment 870]
  + [Comment 868]
  + [Comment 588]
  + [Comment 579]
  + [Comment 564]
  + [Comment 529]
  + [Comment 268]
  + [Comment 243]
  + [Comment 215]
  + [Comment 57]

<Improved Access and Service Roads>
  + [Comment 895]
  + [Comment 882]
  + [Comment 860]
  + [Comment 804]
  + [Comment 803]
  + [Comment 798]
  + [Comment 771]
  + [Comment 660]
  + [Comment 642]
  + [Comment 636]
  + [Comment 596]
  + [Comment 565]
  + [Comment 531]
  + [Comment 505]
  + [Comment 502]
  + [Comment 483]
  + [Comment 435]
  + [Comment 415]
  + [Comment 411]
  + [Comment 407]
  + [Comment 394]
  + [Comment 718]
  + [Comment 352]
  + [Comment 340]
  + [Comment 318]
  + [Comment 293]
  + [Comment 283]
  + [Comment 279]
  + [Comment 258]
  + [Comment 250]
  + [Comment 248]
  + [Comment 172]
  + [Comment 478]
  + [Comment 468]
  + [Comment 467]
  + [Comment 168]
  + [Comment 156]
  + [Comment 153]
  + [Comment 149]
  + [Comment 148]
  + [Comment 128]
  + [Comment 83]
  + [Comment 82]
  + [Comment 69]
  + [Comment 66]

<Enforcing Noise Ordinances and Supporting Mental Health>
  + [Comment 861]
  + [Comment 774]
  + [Comment 695]
  + [Comment 506]
  + [Comment 739]
  + [Comment 379]
  + [Comment 371]
  + [Comment 366]
  + [Comment 174]
  + [Comment 466]
  + [Comment 460]
  + [Comment 139]
  + [Comment 24]

<Comprehensive City Planning and Zoning Strategy>
  + [Comment 850]
  + [Comment 844]
  + [Comment 832]
  + [Comment 821]
  + [Comment 820]
  + [Comment 819]
  + [Comment 657]
  + [Comment 654]
  + [Comment 653]
  + [Comment 650]
  + [Comment 649]
  + [Comment 643]
  + [Comment 631]
  + [Comment 574]
  + [Comment 534]
  + [Comment 528]
  + [Comment 513]
  + [Comment 441]
  + [Comment 440]
  + [Comment 755]
  + [Comment 737]
  + [Comment 731]
  + [Comment 729]
  + [Comment 362]
  + [Comment 301]
  + [Comment 275]
  + [Comment 241]
  + [Comment 192]
  + [Comment 189]
  + [Comment 185]
  + [Comment 183]
  + [Comment 182]
  + [Comment 179]
  + [Comment 178]
  + [Comment 475]
  + [Comment 157]
  + [Comment 111]
  + [Comment 107]
  + [Comment 97]
  + [Comment 88]
  + [Comment 71]
  + [Comment 60]
  + [Comment 58]
  + [Comment 33]
  + [Comment 44]
  + [Comment 18]

<Historic Preservation and Unique Architectural Heritage>
  + [Comment 850]
  + [Comment 654]
  + [Comment 653]
  + [Comment 631]
  + [Comment 440]
  + [Comment 755]
  + [Comment 731]
  + [Comment 275]
  + [Comment 241]
  + [Comment 192]
  + [Comment 183]
  + [Comment 182]
  + [Comment 178]
  + [Comment 157]
  + [Comment 33]
  + [Comment 44]
  + [Comment 18]

<Walkability and Mixed-Use Developments>
  + [Comment 844]
  + [Comment 832]
  + [Comment 820]
  + [Comment 819]
  + [Comment 657]
  + [Comment 659]
  + [Comment 654]
  + [Comment 653]
  + [Comment 650]
  + [Comment 649]
  + [Comment 643]
  + [Comment 631]
  + [Comment 574]
  + [Comment 534]
  + [Comment 513]
  + [Comment 441]
  + [Comment 440]
  + [Comment 755]
  + [Comment 737]
  + [Comment 731]
  + [Comment 700]
  + [Comment 275]
  + [Comment 192]
  + [Comment 189]
  + [Comment 185]
  + [Comment 183]
  + [Comment 178]
  + [Comment 475]
  + [Comment 157]
  + [Comment 97]
  + [Comment 88]
  + [Comment 71]
  + [Comment 60]
  + [Comment 58]
  + [Comment 33]
  + [Comment 44]
  + [Comment 18]

<Immigrant and Refugee Integration>
  + [Comment 820]
  + [Comment 631]
  + [Comment 513]
  + [Comment 440]
  + [Comment 700]
  + [Comment 275]
  + [Comment 189]
  + [Comment 185]
  + [Comment 178]
  + [Comment 111]
  + [Comment 33]
  + [Comment 44]

<Dietary Accommodation>
  + [Comment 545]
  + [Comment 498]
  + [Comment 421]
  + [Comment 419]
  + [Comment 710]
  + [Comment 237]
  + [Comment 236]
  + [Comment 113]

<Inclusive Arts & Culture>
  + [Comment 639]
  + [Comment 635]
  + [Comment 608]
  + [Comment 744]
  + [Comment 256]
  + [Comment 221]
  + [Comment 187]
  + [Comment 138]
  + [Comment 70]
  + [Comment 37]

<Enhancing Park Experience>
  + [Comment 807]
  + [Comment 784]
  + [Comment 766]
  + [Comment 690]
  + [Comment 544]
  + [Comment 526]
  + [Comment 453]
  + [Comment 345]
  + [Comment 326]
  + [Comment 276]
  + [Comment 273]
  + [Comment 251]
  + [Comment 222]
  + [Comment 218]
  + [Comment 214]
  + [Comment 203]
  + [Comment 184]
  + [Comment 163]
  + [Comment 146]
  + [Comment 95]
  + [Comment 76]
  + [Comment 47]

<Downtown Parking Improvement>
  + [Comment 848]
  + [Comment 847]
  + [Comment 603]
  + [Comment 546]
  + [Comment 450]
  + [Comment 392]
  + [Comment 713]
  + [Comment 348]
  + [Comment 322]
  + [Comment 300]
  + [Comment 204]
  + [Comment 104]

<Local Business Development and Support>
  + [Comment 850]
  + [Comment 844]
  + [Comment 820]
  + [Comment 819]
  + [Comment 659]
  + [Comment 654]
  + [Comment 653]
  + [Comment 650]
  + [Comment 649]
  + [Comment 631]
  + [Comment 574]
  + [Comment 513]
  + [Comment 441]
  + [Comment 440]
  + [Comment 412]
  + [Comment 755]
  + [Comment 737]
  + [Comment 731]
  + [Comment 700]
  + [Comment 275]
  + [Comment 189]
  + [Comment 185]
  + [Comment 183]
  + [Comment 182]
  + [Comment 178]
  + [Comment 157]
  + [Comment 88]
  + [Comment 58]
  + [Comment 33]
  + [Comment 44]
  + [Comment 18]

<Educate about drug risks>
  + [Comment 742]
  + [Comment 709]
  + [Comment 10]

<Voter Support>
  + [Comment 769]
  + [Comment 610]
  + [Comment 85]
  + [Comment 12]

<Revenue Generation>
  + [Comment 129]
  + [Comment 85]
  + [Comment 13]

<Occupational Tax for Homeless Support>
  + [Comment 827]
  + [Comment 782]
  + [Comment 572]
  + [Comment 535]
  + [Comment 522]
  + [Comment 436]
  + [Comment 409]
  + [Comment 404]
  + [Comment 395]
  + [Comment 341]
  + [Comment 266]
  + [Comment 269]
  + [Comment 267]
  + [Comment 260]
  + [Comment 232]
  + [Comment 160]
  + [Comment 140]
  + [Comment 147]
  + [Comment 112]
  + [Comment 34]
  + [Comment 4]

<Term Limits for Fair Representation>
  + [Comment 839]
  + [Comment 572]
  + [Comment 458]
  + [Comment 438]
  + [Comment 140]
  + [Comment 145]
  + [Comment 127]
  + [Comment 126]
  + [Comment 67]
  + [Comment 61]

<Poverty Reduction and Elderly Support>
  + [Comment 890]
  + [Comment 822]
  + [Comment 790]
  + [Comment 782]
  + [Comment 684]
  + [Comment 641]
  + [Comment 634]
  + [Comment 590]
  + [Comment 589]
  + [Comment 587]
  + [Comment 580]
  + [Comment 572]
  + [Comment 535]
  + [Comment 522]
  + [Comment 452]
  + [Comment 437]
  + [Comment 436]
  + [Comment 409]
  + [Comment 749]
  + [Comment 341]
  + [Comment 297]
  + [Comment 266]
  + [Comment 267]
  + [Comment 265]
  + [Comment 260]
  + [Comment 244]
  + [Comment 232]
  + [Comment 220]
  + [Comment 213]
  + [Comment 473]
  + [Comment 160]
  + [Comment 144]
  + [Comment 112]
  + [Comment 114]
  + [Comment 50]
  + [Comment 34]
  + [Comment 4]

<Improved Peak Hour Traffic Flow>
  + [Comment 895]
  + [Comment 882]
  + [Comment 866]
  + [Comment 860]
  + [Comment 805]
  + [Comment 804]
  + [Comment 803]
  + [Comment 798]
  + [Comment 771]
  + [Comment 660]
  + [Comment 642]
  + [Comment 636]
  + [Comment 596]
  + [Comment 595]
  + [Comment 585]
  + [Comment 565]
  + [Comment 531]
  + [Comment 505]
  + [Comment 502]
  + [Comment 486]
  + [Comment 484]
  + [Comment 483]
  + [Comment 435]
  + [Comment 415]
  + [Comment 411]
  + [Comment 407]
  + [Comment 394]
  + [Comment 718]
  + [Comment 352]
  + [Comment 340]
  + [Comment 318]
  + [Comment 311]
  + [Comment 293]
  + [Comment 283]
  + [Comment 279]
  + [Comment 258]
  + [Comment 250]
  + [Comment 227]
  + [Comment 172]
  + [Comment 478]
  + [Comment 468]
  + [Comment 467]
  + [Comment 168]
  + [Comment 156]
  + [Comment 153]
  + [Comment 149]
  + [Comment 148]
  + [Comment 131]
  + [Comment 128]
  + [Comment 83]
  + [Comment 82]
  + [Comment 69]
  + [Comment 66]

<Diversifying Food Options>
  + [Comment 774]
  + [Comment 692]
  + [Comment 667]
  + [Comment 662]
  + [Comment 538]
  + [Comment 196]
  + [Comment 23]

<Fostering Inclusivity and Economic Growth>
  + [Comment 775]
  + [Comment 695]
  + [Comment 538]
  + [Comment 398]
  + [Comment 371]
  + [Comment 196]
  + [Comment 177]
  + [Comment 174]
  + [Comment 170]
  + [Comment 87]
  + [Comment 86]
  + [Comment 20]

<Family-Centered Activities and Entertainment>
  + [Comment 844]
  + [Comment 832]
  + [Comment 819]
  + [Comment 657]
  + [Comment 653]
  + [Comment 650]
  + [Comment 643]
  + [Comment 577]
  + [Comment 574]
  + [Comment 534]
  + [Comment 513]
  + [Comment 440]
  + [Comment 755]
  + [Comment 737]
  + [Comment 731]
  + [Comment 729]
  + [Comment 700]
  + [Comment 362]
  + [Comment 275]
  + [Comment 189]
  + [Comment 185]
  + [Comment 183]
  + [Comment 182]
  + [Comment 178]
  + [Comment 475]
  + [Comment 157]
  + [Comment 111]
  + [Comment 71]
  + [Comment 60]
  + [Comment 58]
  + [Comment 33]
  + [Comment 18]

<Green Spaces and Shaded Areas>
  + [Comment 832]
  + [Comment 821]
  + [Comment 819]
  + [Comment 659]
  + [Comment 653]
  + [Comment 643]
  + [Comment 577]
  + [Comment 534]
  + [Comment 528]
  + [Comment 513]
  + [Comment 441]
  + [Comment 440]
  + [Comment 755]
  + [Comment 737]
  + [Comment 729]
  + [Comment 700]
  + [Comment 362]
  + [Comment 275]
  + [Comment 192]
  + [Comment 178]
  + [Comment 475]
  + [Comment 111]
  + [Comment 65]
  + [Comment 60]
  + [Comment 58]
  + [Comment 44]
  + [Comment 18]

<Enhancing Greenway Experience>
  + [Comment 807]
  + [Comment 784]
  + [Comment 766]
  + [Comment 690]
  + [Comment 544]
  + [Comment 526]
  + [Comment 453]
  + [Comment 345]
  + [Comment 276]
  + [Comment 273]
  + [Comment 251]
  + [Comment 222]
  + [Comment 218]
  + [Comment 214]
  + [Comment 184]
  + [Comment 163]
  + [Comment 146]
  + [Comment 95]
  + [Comment 76]
  + [Comment 47]

<Fostering Workforce Development>
  + [Comment 885]
  + [Comment 724]
  + [Comment 470]

<Bridging the Digital Divide>
  + [Comment 863]
  + [Comment 796]
  + [Comment 724]
  + [Comment 470]
  + [Comment 115]
  + [Comment 99]

<Driving Competition and Improving Services>
  + [Comment 872]
  + [Comment 809]
  + [Comment 523]
  + [Comment 494]
  + [Comment 423]
  + [Comment 715]
  + [Comment 247]
  + [Comment 295]
  + [Comment 292]
  + [Comment 228]
  + [Comment 201]
  + [Comment 200]
  + [Comment 155]
  + [Comment 135]
  + [Comment 64]

<Sustainable Recycling>
  + [Comment 816]
  + [Comment 815]
  + [Comment 673]
  + [Comment 578]
  + [Comment 536]
  + [Comment 414]
  + [Comment 302]
  + [Comment 208]

<Enhanced BG buses>
  + [Comment 870]
  + [Comment 868]
  + [Comment 588]
  + [Comment 579]
  + [Comment 529]
  + [Comment 268]
  + [Comment 243]
  + [Comment 215]
  + [Comment 57]

<Affordable Housing and Property Ownership>
  + [Comment 890]
  + [Comment 875]
  + [Comment 822]
  + [Comment 581]
  + [Comment 572]
  + [Comment 567]
  + [Comment 535]
  + [Comment 499]
  + [Comment 452]
  + [Comment 436]
  + [Comment 409]
  + [Comment 405]
  + [Comment 265]
  + [Comment 260]
  + [Comment 232]
  + [Comment 223]
  + [Comment 220]
  + [Comment 213]
  + [Comment 176]
  + [Comment 459]
  + [Comment 160]
  + [Comment 140]
  + [Comment 147]
  + [Comment 144]
  + [Comment 112]
  + [Comment 114]
  + [Comment 61]
  + [Comment 55]
  + [Comment 50]
  + [Comment 34]
  + [Comment 30]

<Improved Driver Education and Enforcement>
  + [Comment 860]
  + [Comment 859]
  + [Comment 804]
  + [Comment 803]
  + [Comment 798]
  + [Comment 660]
  + [Comment 642]
  + [Comment 636]
  + [Comment 596]
  + [Comment 585]
  + [Comment 565]
  + [Comment 531]
  + [Comment 486]
  + [Comment 485]
  + [Comment 484]
  + [Comment 483]
  + [Comment 435]
  + [Comment 415]
  + [Comment 407]
  + [Comment 714]
  + [Comment 352]
  + [Comment 311]
  + [Comment 283]
  + [Comment 282]
  + [Comment 258]
  + [Comment 250]
  + [Comment 478]
  + [Comment 156]
  + [Comment 154]
  + [Comment 128]
  + [Comment 122]
  + [Comment 110]
  + [Comment 69]
  + [Comment 66]

<Enhancing Aesthetic Appeal>
  + [Comment 861]
  + [Comment 775]
  + [Comment 695]
  + [Comment 488]
  + [Comment 371]
  + [Comment 366]
  + [Comment 174]
  + [Comment 466]
  + [Comment 171]
  + [Comment 36]

<Implement stricter opioid regulations>
  + [Comment 894]
  + [Comment 846]
  + [Comment 686]
  + [Comment 683]
  + [Comment 679]
  + [Comment 678]
  + [Comment 500]
  + [Comment 753]
  + [Comment 752]
  + [Comment 742]
  + [Comment 709]
  + [Comment 336]
  + [Comment 245]
  + [Comment 11]
  + [Comment 10]

<Physical Activity for Health>
  + [Comment 869]
  + [Comment 422]
  + [Comment 421]
  + [Comment 237]
  + [Comment 236]

<Healthier Environment>
  + [Comment 382]
  + [Comment 381]

<Grocery Store Competition>
  + [Comment 648]
  + [Comment 452]
  + [Comment 405]
  + [Comment 265]
  + [Comment 140]

<Safer and More Convenient Roads>
  + [Comment 895]
  + [Comment 882]
  + [Comment 860]
  + [Comment 805]
  + [Comment 804]
  + [Comment 798]
  + [Comment 771]
  + [Comment 660]
  + [Comment 642]
  + [Comment 596]
  + [Comment 595]
  + [Comment 565]
  + [Comment 531]
  + [Comment 505]
  + [Comment 502]
  + [Comment 484]
  + [Comment 483]
  + [Comment 435]
  + [Comment 415]
  + [Comment 411]
  + [Comment 394]
  + [Comment 718]
  + [Comment 352]
  + [Comment 340]
  + [Comment 339]
  + [Comment 318]
  + [Comment 293]
  + [Comment 283]
  + [Comment 279]
  + [Comment 258]
  + [Comment 250]
  + [Comment 248]
  + [Comment 227]
  + [Comment 209]
  + [Comment 478]
  + [Comment 468]
  + [Comment 467]
  + [Comment 168]
  + [Comment 156]
  + [Comment 150]
  + [Comment 153]
  + [Comment 149]
  + [Comment 148]
  + [Comment 131]
  + [Comment 128]
  + [Comment 83]
  + [Comment 82]
  + [Comment 69]

<Enhancing Scenic Beauty>
  + [Comment 861]
  + [Comment 695]
  + [Comment 671]
  + [Comment 524]
  + [Comment 488]
  + [Comment 371]
  + [Comment 366]
  + [Comment 466]
  + [Comment 87]
  + [Comment 36]

<Food Access Improvement>
  + [Comment 696]
  + [Comment 640]
  + [Comment 548]
  + [Comment 446]
  + [Comment 759]
  + [Comment 706]
  + [Comment 77]
  + [Comment 73]
  + [Comment 32]

<Sign and Green Space Regulation>
  + [Comment 847]
  + [Comment 558]
  + [Comment 550]
  + [Comment 533]
  + [Comment 455]
  + [Comment 191]
  + [Comment 32]

<Existing Housing Renovations and Improvements>
  + [Comment 850]
  + [Comment 821]
  + [Comment 819]
  + [Comment 654]
  + [Comment 653]
  + [Comment 649]
  + [Comment 631]
  + [Comment 528]
  + [Comment 429]
  + [Comment 416]
  + [Comment 731]
  + [Comment 241]
  + [Comment 192]
  + [Comment 183]
  + [Comment 179]
  + [Comment 178]
  + [Comment 107]
  + [Comment 65]
  + [Comment 33]
  + [Comment 44]
  + [Comment 18]

<Enhancing Waterfront Experience>
  + [Comment 784]
  + [Comment 766]
  + [Comment 453]
  + [Comment 345]
  + [Comment 276]
  + [Comment 273]
  + [Comment 251]
  + [Comment 222]
  + [Comment 218]
  + [Comment 214]
  + [Comment 163]
  + [Comment 47]

<Ensuring Fair Cable TV Services>
  + [Comment 874]
  + [Comment 247]
  + [Comment 292]
  + [Comment 200]
  + [Comment 155]
  + [Comment 135]

<Improving Internet Speeds and Competition>
  + [Comment 872]
  + [Comment 773]
  + [Comment 523]
  + [Comment 494]
  + [Comment 423]
  + [Comment 247]
  + [Comment 295]
  + [Comment 292]
  + [Comment 228]
  + [Comment 201]
  + [Comment 200]
  + [Comment 155]
  + [Comment 135]
  + [Comment 64]

<Economic Growth>
  + [Comment 129]
  + [Comment 93]
  + [Comment 85]

<Accessible Recycling>
  + [Comment 816]
  + [Comment 815]
  + [Comment 689]
  + [Comment 673]
  + [Comment 578]
  + [Comment 557]
  + [Comment 536]
  + [Comment 414]
  + [Comment 308]
  + [Comment 302]
  + [Comment 208]

<Empowering Blue Collar Workers>
  + [Comment 775]
  + [Comment 774]
  + [Comment 466]
  + [Comment 171]
  + [Comment 170]
  + [Comment 159]
  + [Comment 36]
  + [Comment 24]

<Educated and Cultured Community>
  + [Comment 891]
  + [Comment 693]
  + [Comment 639]
  + [Comment 635]
  + [Comment 608]
  + [Comment 514]
  + [Comment 410]
  + [Comment 748]
  + [Comment 744]
  + [Comment 347]
  + [Comment 314]
  + [Comment 287]
  + [Comment 256]
  + [Comment 231]
  + [Comment 221]
  + [Comment 190]
  + [Comment 187]
  + [Comment 181]
  + [Comment 180]
  + [Comment 138]
  + [Comment 72]
  + [Comment 70]
  + [Comment 63]
  + [Comment 56]
  + [Comment 37]
  + [Comment 15]

<Driving Competition and Improving Internet Access>
  + [Comment 872]
  + [Comment 809]
  + [Comment 773]
  + [Comment 523]
  + [Comment 494]
  + [Comment 423]
  + [Comment 247]
  + [Comment 295]
  + [Comment 292]
  + [Comment 228]
  + [Comment 201]
  + [Comment 200]
  + [Comment 155]
  + [Comment 135]
  + [Comment 64]

<Deter Littering>
  + [Comment 689]
  + [Comment 673]
  + [Comment 606]
  + [Comment 578]
  + [Comment 557]
  + [Comment 414]
  + [Comment 308]
  + [Comment 208]

```
